# Create dataset


## Quarterly hours for Macrozone Nord

We start considering the Macrozone NORD

In [1]:
import sys
print(sys.executable)

c:\Users\samueledelia\.conda\envs\myenv\python.exe


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_nord = qh[qh['MACROZONA'] == 'NORD']
qh_nord = qh_nord.drop(columns = ['MACROZONA'])

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [3]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_ENERGY_BALANCE" 
power = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power = power.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
power = power[['ENERGY_BALANCE_GWh', 'ORAINI', 'ENERGY_SOURCE']]
# Convert ORAINI to datetime using the correct method
power['ORAINI'] = pd.to_datetime(power['ORAINI'], format='%Y%m%d%H%M')
power.set_index('ORAINI', inplace=True)
thermo = utils.filter_and_rename_energy_source(power, 'Thermal', 'THERMO_GWH')
wind = utils.filter_and_rename_energy_source(power, 'Wind', 'WIND_GWH')
solar = utils.filter_and_rename_energy_source(power, 'Photovoltaic', 'SOLAR_GWH')
hydro = utils.filter_and_rename_energy_source(power, 'Hydro', 'HYDRO_GWH')
geothermal = utils.filter_and_rename_energy_source(power, 'Geothermal', 'GEO_GWH')
forex = utils.filter_and_rename_energy_source(power, 'Net Foreign Exchange', 'FOREX_GWH')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [4]:
# Resample the data to 15-minute intervals and divide by 4
wind = wind.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
wind['WIND_GWH'] = wind['WIND_GWH'] / 4
wind['WIND_MWH'] = wind['WIND_GWH'] * 1000
# Drop the original hourly GWh column
wind = wind.drop(columns=['WIND_GWH'])

wind

,WIND_MWH
ORAINI,
2019-01-01 00:00:00,1275.0
2019-01-01 00:15:00,1275.0
2019-01-01 00:30:00,1275.0
2019-01-01 00:45:00,1275.0
2019-01-01 01:00:00,1257.5
...,...
2024-12-17 22:00:00,427.5
2024-12-17 22:15:00,427.5
2024-12-17 22:30:00,427.5


In [5]:
# Resample the data to 15-minute intervals and divide by 4
solar = solar.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
solar['SOLAR_GWH'] = solar['SOLAR_GWH'] / 4
solar['SOLAR_MWH'] = solar['SOLAR_GWH'] * 1000
# Drop the original hourly GWh column
solar = solar.drop(columns=['SOLAR_GWH'])

solar

,SOLAR_MWH
ORAINI,
2019-01-01 00:00:00,0.0
2019-01-01 00:15:00,0.0
2019-01-01 00:30:00,0.0
2019-01-01 00:45:00,0.0
2019-01-01 01:00:00,0.0
...,...
2024-12-17 22:00:00,0.0
2024-12-17 22:15:00,0.0
2024-12-17 22:30:00,0.0


In [6]:
# Resample the data to 15-minute intervals and divide by 4
thermo = thermo.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
thermo['THERMO_GWH'] = thermo['THERMO_GWH'] / 4
thermo['THERMO_MWH'] = thermo['THERMO_GWH'] * 1000
# Drop the original hourly GWh column
thermo = thermo.drop(columns=['THERMO_GWH'])

thermo

,THERMO_MWH
ORAINI,
2019-01-01 00:00:00,3337.5
2019-01-01 00:15:00,3337.5
2019-01-01 00:30:00,3337.5
2019-01-01 00:45:00,3337.5
2019-01-01 01:00:00,3272.5
...,...
2024-12-17 22:00:00,5342.5
2024-12-17 22:15:00,5342.5
2024-12-17 22:30:00,5342.5


In [7]:
# Resample the data to 15-minute intervals and divide by 4
hydro = hydro.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
hydro['HYDRO_GWH'] = hydro['HYDRO_GWH'] / 4
hydro['HYDRO_MWH'] = hydro['HYDRO_GWH'] * 1000
# Drop the original hourly GWh column
hydro = hydro.drop(columns=['HYDRO_GWH'])

hydro

,HYDRO_MWH
ORAINI,
2019-01-01 00:00:00,515.0
2019-01-01 00:15:00,515.0
2019-01-01 00:30:00,515.0
2019-01-01 00:45:00,515.0
2019-01-01 01:00:00,437.5
...,...
2024-12-17 22:00:00,550.0
2024-12-17 22:15:00,550.0
2024-12-17 22:30:00,550.0


In [8]:
# Resample the data to 15-minute intervals and divide by 4
geothermal = geothermal.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
geothermal['GEO_GWH'] = geothermal['GEO_GWH'] / 4
geothermal['GEO_MWH'] = geothermal['GEO_GWH'] * 1000
# Drop the original hourly GWh column
geothermal = geothermal.drop(columns=['GEO_GWH'])

geothermal

,GEO_MWH
ORAINI,
2019-01-01 00:00:00,167.5
2019-01-01 00:15:00,167.5
2019-01-01 00:30:00,167.5
2019-01-01 00:45:00,167.5
2019-01-01 01:00:00,167.5
...,...
2024-12-17 22:00:00,150.0
2024-12-17 22:15:00,150.0
2024-12-17 22:30:00,150.0


In [9]:
## Resample the data to 15-minute intervals and divide by 4
#forex = forex.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
#forex['FOREX_GWH'] = forex['FOREX_GWH'] / 4
#forex['FOREX_MWH'] = forex['FOREX_GWH'] * 1000
# Drop the original hourly GWh column
#forex = forex.drop(columns=['FOREX_GWH'])

#forex

In [10]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_PHYSICAL_FOREIGN_FLOW" 
physical_forex = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
physical_forex = physical_forex.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
physical_forex = physical_forex[['PHYSICAL_FOREIGN_FLOW_MW', 'ORAINI', 'COUNTRY']]
# Convert ORAINI to datetime using the correct method
physical_forex['ORAINI'] = pd.to_datetime(physical_forex['ORAINI'], format='%Y%m%d%H%M')
physical_forex.set_index('ORAINI', inplace=True)

physical_forex

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,PHYSICAL_FOREIGN_FLOW_MW,COUNTRY
ORAINI,,
2019-01-01 00:00:00,156.0,Austria
2019-01-01 00:00:00,-142.0,Corsica
2019-01-01 00:00:00,-321.0,France
2019-01-01 00:00:00,-313.0,Greece
2019-01-01 00:00:00,-80.0,Malta
...,...,...
2024-12-17 23:00:00,2889.0,France
2024-12-17 23:00:00,403.0,Greece
2024-12-17 23:00:00,-105.0,Malta


In [11]:
austria_physical = utils.process_foreign_exchange(physical_forex, 'Austria')
corsica_physical = utils.process_foreign_exchange(physical_forex, 'Corsica')
france_physical = utils.process_foreign_exchange(physical_forex, 'France')
greece_physical = utils.process_foreign_exchange(physical_forex, 'Greece')
malta_physical = utils.process_foreign_exchange(physical_forex, 'Malta')
slovenia_physical = utils.process_foreign_exchange(physical_forex, 'Slovenia')
switzerland_physical = utils.process_foreign_exchange(physical_forex, 'Switzerland')
montenegno_physical = utils.process_foreign_exchange(physical_forex, 'Montenegno')

In [12]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME,FLOWQUARTER,TIMERESOLUTION
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097,NaN,None
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097,NaN,None
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097,NaN,None
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100,NaN,None
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-13 19:00:00,43210.855,44069.484,3675.566,7793.277,23323.551,1174.003,2364.969,2613.840,0.0,None,...,477.000,0.0,0.0,0.0,0.0,44010.515,43510.555,2024-12-12 13:10:49.317,20.0,PT60
2024-12-13 20:00:00,40943.994,40943.994,3403.500,7425.759,21669.275,1153.128,2320.965,2515.856,0.0,None,...,784.000,0.0,0.0,0.0,625.0,40172.271,40943.994,2024-12-12 13:10:49.317,21.0,PT60
2024-12-13 21:00:00,37181.938,37181.938,3108.442,6828.312,20012.009,1075.712,2164.911,2326.693,0.0,None,...,1623.000,0.0,0.0,0.0,533.0,36149.383,37181.938,2024-12-12 13:10:49.317,22.0,PT60


In [13]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mgp_volumes_nord.rename(columns={"NORD_PURCHASES": "MGP_NORD_PURCHASES", "NORD_SALES": "MGP_NORD_SALES"}, inplace=True)
mgp_volumes_nord = mgp_volumes_nord[~mgp_volumes_nord.index.duplicated(keep='first')]
mgp_volumes_nord = mgp_volumes_nord.resample('15min').ffill()/4


mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
mi1_volumes = mi1_volumes.drop(columns=['MARKET'])
mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mi1_volumes_nord.rename(columns={"NORD_PURCHASES": "MI1_NORD_PURCHASES", "NORD_SALES": "MI1_NORD_SALES"}, inplace=True)
mi1_volumes_nord = mi1_volumes_nord[~mi1_volumes_nord.index.duplicated(keep='first')]
mi1_volumes_nord = mi1_volumes_nord.resample('15min').ffill()/4

In [14]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SCHEDULED_INTERNAL_EXCHANGE"
scheduled_internal = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
scheduled_internal = scheduled_internal.sort_values(by='ORAINI')

# Filter the DataFrame for the relevant columns and rows
scheduled_internal = scheduled_internal[['SCHEDULED_INTERNAL_EXCHANGE_MW', 'ORAINI', 'FLOW_FROM', 'FLOW_TO']]
# Convert ORAINI to datetime using the correct method
scheduled_internal['ORAINI'] = pd.to_datetime(scheduled_internal['ORAINI'], format='%Y%m%d%H%M')
scheduled_internal.set_index('ORAINI', inplace=True)
scheduled_internal = scheduled_internal[(scheduled_internal['FLOW_FROM'] == 'CNOR') & (scheduled_internal['FLOW_TO'] == 'NORD') ]
scheduled_internal = scheduled_internal.drop(columns=['FLOW_FROM','FLOW_TO'])
scheduled_internal_nord = scheduled_internal.resample('15min').ffill()/4

scheduled_internal_nord

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,SCHEDULED_INTERNAL_EXCHANGE_MW
ORAINI,
2019-01-01 00:00:00,158.7875
2019-01-01 00:15:00,158.7875
2019-01-01 00:30:00,158.7875
2019-01-01 00:45:00,158.7875
2019-01-01 01:00:00,235.1475
...,...
2024-12-17 11:00:00,140.4525
2024-12-17 11:15:00,140.4525
2024-12-17 11:30:00,140.4525


In [15]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_TOTAL_LOAD_QH" 
total_load = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
total_load = total_load.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
total_load = total_load[['ZONA', 'TOTAL_LOAD_MW', 'FORECAST_TOTAL_LOAD_MW', 'ORAINI']]
# Convert ORAINI to datetime using the correct method
total_load['ORAINI'] = pd.to_datetime(total_load['ORAINI'], format='%Y%m%d%H%M')
total_load.set_index('ORAINI', inplace=True)
subset_zonas = ['NORD']
mnord_load = utils.filter_load_by_zona(total_load, subset_zonas)
mnord_load = mnord_load.drop(columns=['ZONA'])
mnord_load

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,11776.405,12085.191
2019-01-01 00:15:00,11723.369,12010.132
2019-01-01 00:30:00,11771.133,11984.692
2019-01-01 00:45:00,11740.606,11935.066
2019-01-01 01:00:00,11559.374,11825.838
...,...,...
2024-12-17 22:45:00,18651.609,19221.792
2024-12-17 23:00:00,18205.958,18728.653
2024-12-17 23:15:00,17537.380,18039.983


Total and forecast load ENTSOE

In [16]:
# Load the data
df_load_ensoe_path = r'C:\imbalance_forecast\data\df_load_entsoe.csv'
df_load_ensoe_path = pd.read_csv(df_load_ensoe_path, index_col=0, parse_dates=True)

# Access the relevant columns
mnord_load_entsoe = df_load_ensoe_path[['DAY_AHEAD_TOTAL_LOAD_MW_NORD', 'ACTUAL_TOTAL_LOAD_MW_NORD']]

# Check for duplicate timestamps
duplicates = mnord_load_entsoe.index.duplicated(keep=False)

if duplicates.any():
    # Handle duplicates: average values for duplicate timestamps
    mnord_load_entsoe = mnord_load_entsoe.groupby(mnord_load_entsoe.index).mean()

# Now resample the data
mnord_load_entsoe = mnord_load_entsoe.resample('15min').ffill() / 4

# Display the result
mnord_load_entsoe

,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,
2019-01-01 00:00:00,2777.50,2877.75
2019-01-01 00:15:00,2777.50,2877.75
2019-01-01 00:30:00,2777.50,2877.75
2019-01-01 00:45:00,2777.50,2877.75
2019-01-01 01:00:00,2668.75,2759.75
...,...,...
2024-11-27 06:00:00,4882.00,4885.00
2024-11-27 06:15:00,4882.00,4885.00
2024-11-27 06:30:00,4882.00,4885.00


Lag SBIL

In [17]:
lags = [-1, -4, -96]

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(qh_nord['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged

,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag96
ORAINI,,,
2019-01-01 00:00:00,NaN,NaN,NaN
2019-01-01 00:15:00,-47.312,NaN,NaN
2019-01-01 00:30:00,-30.343,NaN,NaN
2019-01-01 00:45:00,-20.311,NaN,NaN
2019-01-01 01:00:00,-13.934,-47.312,NaN
...,...,...,...
2024-12-18 15:15:00,161.331,105.467,36.691
2024-12-18 15:30:00,91.954,72.953,-5.659
2024-12-18 15:45:00,25.418,24.432,-96.750


In [18]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [qh_nord, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, austria_physical, france_physical, slovenia_physical, switzerland_physical, scheduled_internal_nord, mgp_volumes_nord, mi1_volumes_nord, mnord_load, mnord_load_entsoe]
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

df_nord = df_nord[df_nord.index <= '2024-03-26']
df_nord = df_nord[df_nord.index >= '2019-01-03']
df_nord.head(35)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag96,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,AUSTRIA_MWQH,...,SWITZERLAND_MWQH,SCHEDULED_INTERNAL_EXCHANGE_MW,MGP_NORD_PURCHASES,MGP_NORD_SALES,MI1_NORD_PURCHASES,MI1_NORD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-03 00:00:00,-1.847,63.465,-156.567,38.978,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,13261.060,13912.337,3194.50,3126.25
2019-01-03 00:15:00,-11.944,-1.847,-63.103,56.932,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,12996.362,13697.814,3194.50,3126.25
2019-01-03 00:30:00,0.336,-11.944,16.441,122.707,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,12869.871,13592.390,3194.50,3126.25
2019-01-03 00:45:00,55.911,0.336,63.465,174.893,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,12624.143,13419.621,3194.50,3126.25
2019-01-03 01:00:00,-17.541,55.911,-1.847,99.216,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12234.627,13132.506,2954.75,2918.50
2019-01-03 01:15:00,-46.081,-17.541,-11.944,104.522,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12286.212,12814.419,2954.75,2918.50
2019-01-03 01:30:00,-21.225,-46.081,0.336,130.053,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12437.186,12880.788,2954.75,2918.50
2019-01-03 01:45:00,29.607,-21.225,55.911,136.617,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12243.895,12746.228,2954.75,2918.50
2019-01-03 02:00:00,-46.653,29.607,-17.541,106.558,1220.0,3150.0,0.0,405.0,170.0,43.25,...,388.00,298.3425,2990.44050,1977.44700,108.58025,106.28475,12027.606,12552.932,2827.25,2825.25


In [19]:
df_nord.tail(25)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag96,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,AUSTRIA_MWQH,...,SWITZERLAND_MWQH,SCHEDULED_INTERNAL_EXCHANGE_MW,MGP_NORD_PURCHASES,MGP_NORD_SALES,MI1_NORD_PURCHASES,MI1_NORD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2024-03-25 18:00:00,23.524,48.439,94.581,35.055,740.0,5087.5,7.5,2540.0,155.0,-42.00,...,225.00,1.22,5282.15800,4578.15225,359.89700,218.58050,22601.901,22091.040,5258.75,5267.25
2024-03-25 18:15:00,58.200,23.524,82.664,310.856,740.0,5087.5,7.5,2540.0,155.0,-42.00,...,225.00,1.22,5282.15800,4578.15225,359.89700,218.58050,23104.769,22359.546,5258.75,5267.25
2024-03-25 18:30:00,-144.780,58.200,25.212,208.539,740.0,5087.5,7.5,2540.0,155.0,-42.00,...,225.00,1.22,5282.15800,4578.15225,359.89700,218.58050,23532.443,22515.042,5258.75,5267.25
2024-03-25 18:45:00,-169.037,-144.780,48.439,105.929,740.0,5087.5,7.5,2540.0,155.0,-42.00,...,225.00,1.22,5282.15800,4578.15225,359.89700,218.58050,23775.259,22772.961,5258.75,5267.25
2024-03-25 19:00:00,-139.162,-169.037,23.524,-35.018,815.0,5007.5,0.0,2767.5,155.0,-43.75,...,258.25,1.22,5355.09375,4636.16025,355.74075,231.28575,23696.325,22856.789,5396.00,5351.25
2024-03-25 19:15:00,-126.473,-139.162,58.200,-58.434,815.0,5007.5,0.0,2767.5,155.0,-43.75,...,258.25,1.22,5355.09375,4636.16025,355.74075,231.28575,23718.077,23094.998,5396.00,5351.25
2024-03-25 19:30:00,-90.585,-126.473,-144.780,-50.204,815.0,5007.5,0.0,2767.5,155.0,-43.75,...,258.25,1.22,5355.09375,4636.16025,355.74075,231.28575,23571.325,23160.790,5396.00,5351.25
2024-03-25 19:45:00,50.697,-90.585,-169.037,-64.585,815.0,5007.5,0.0,2767.5,155.0,-43.75,...,258.25,1.22,5355.09375,4636.16025,355.74075,231.28575,23348.847,23128.659,5396.00,5351.25
2024-03-25 20:00:00,-101.413,50.697,-139.162,-37.400,902.5,4755.0,0.0,2035.0,155.0,-21.00,...,514.75,1.22,5160.57550,4316.72225,299.43475,109.42350,23034.077,22897.287,5157.00,5089.50


In [20]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord.to_csv(filepath)


<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\4209121318.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord.csv')


### Add E. portfolio

In [21]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = """
SELECT * FROM POWER_UNBALANCE
"""
power_curve = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power_curve = power_curve.sort_values(by='TIMESTAMP')

power_curve

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TIMESTAMP,UNBALANCE_kW,SOURCE_ZONE
0,2023-11-01 01:00:23.830,-629.990,EOLICO_CSUD
1,2023-11-01 01:00:23.830,-208.450,EOLICO_SUD
2,2023-11-01 01:00:23.830,-781.500,IDRO-NON-PROGRAMMABILE_H_CNOR
3,2023-11-01 01:00:23.830,104.480,IDRO-NON-PROGRAMMABILE_H_CSUD
4,2023-11-01 01:00:23.830,2238.280,IDRO-NON-PROGRAMMABILE_H_NORD
...,...,...,...
7052760,2024-12-18 16:41:42.977,-153.390,EOLICO_CSUD
7052759,2024-12-18 16:41:42.977,505.390,EOLICO_CALA
7052775,2024-12-18 16:41:42.977,-80.750,SOLARE_SICI
7052766,2024-12-18 16:41:42.977,12.340,IDRO-NON-PROGRAMMABILE_H24_CNOR


In [22]:
power_curve = power_curve.pivot(index='TIMESTAMP', columns='SOURCE_ZONE', values='UNBALANCE_kW')

# Rename columns for clarity (optional, if needed)
power_curve.columns = [f"UNBALANCE_{col}" for col in power_curve.columns]

# Display the resulting DataFrame
power_curve = power_curve.resample('15min').sum()

power_curve = power_curve[["UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD", "UNBALANCE_IDRO-PROGRAMMABILE_NORD", "UNBALANCE_SOLARE_NORD"]]

power_curve = power_curve.rename_axis("ORAINI")

power_curve

,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,
2023-11-01 01:00:00,0.00,3137.91,0.000
2023-11-01 01:15:00,0.00,3124.79,0.000
2023-11-01 01:30:00,0.00,3047.20,0.000
2023-11-01 01:45:00,0.00,3137.53,0.000
2023-11-01 02:00:00,0.00,3056.03,0.000
...,...,...,...
2024-12-18 15:30:00,765.41,-1394.13,-5836.425
2024-12-18 15:45:00,9785.63,-1890.04,-25760.685
2024-12-18 16:00:00,1278.14,-60033.70,1727.324


In [23]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [qh_nord, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, austria_physical, france_physical, slovenia_physical, switzerland_physical, scheduled_internal_nord, mgp_volumes_nord, mi1_volumes_nord, mnord_load, mnord_load_entsoe, power_curve]
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord_port = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

df_nord_port = df_nord_port[df_nord_port.index <= '2024-11-26']
df_nord_port = df_nord_port[df_nord_port.index >= '2019-01-03']
df_nord_port.head(35)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag96,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,AUSTRIA_MWQH,...,MGP_NORD_SALES,MI1_NORD_PURCHASES,MI1_NORD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-03 00:00:00,-1.847,63.465,-156.567,38.978,1402.5,3242.5,0.0,407.5,167.5,39.25,...,2003.79250,151.94800,106.55450,13261.060,13912.337,3194.50,3126.25,NaN,NaN,NaN
2019-01-03 00:15:00,-11.944,-1.847,-63.103,56.932,1402.5,3242.5,0.0,407.5,167.5,39.25,...,2003.79250,151.94800,106.55450,12996.362,13697.814,3194.50,3126.25,NaN,NaN,NaN
2019-01-03 00:30:00,0.336,-11.944,16.441,122.707,1402.5,3242.5,0.0,407.5,167.5,39.25,...,2003.79250,151.94800,106.55450,12869.871,13592.390,3194.50,3126.25,NaN,NaN,NaN
2019-01-03 00:45:00,55.911,0.336,63.465,174.893,1402.5,3242.5,0.0,407.5,167.5,39.25,...,2003.79250,151.94800,106.55450,12624.143,13419.621,3194.50,3126.25,NaN,NaN,NaN
2019-01-03 01:00:00,-17.541,55.911,-1.847,99.216,1360.0,3160.0,0.0,395.0,170.0,40.50,...,1992.72250,129.96975,134.68950,12234.627,13132.506,2954.75,2918.50,NaN,NaN,NaN
2019-01-03 01:15:00,-46.081,-17.541,-11.944,104.522,1360.0,3160.0,0.0,395.0,170.0,40.50,...,1992.72250,129.96975,134.68950,12286.212,12814.419,2954.75,2918.50,NaN,NaN,NaN
2019-01-03 01:30:00,-21.225,-46.081,0.336,130.053,1360.0,3160.0,0.0,395.0,170.0,40.50,...,1992.72250,129.96975,134.68950,12437.186,12880.788,2954.75,2918.50,NaN,NaN,NaN
2019-01-03 01:45:00,29.607,-21.225,55.911,136.617,1360.0,3160.0,0.0,395.0,170.0,40.50,...,1992.72250,129.96975,134.68950,12243.895,12746.228,2954.75,2918.50,NaN,NaN,NaN
2019-01-03 02:00:00,-46.653,29.607,-17.541,106.558,1220.0,3150.0,0.0,405.0,170.0,43.25,...,1977.44700,108.58025,106.28475,12027.606,12552.932,2827.25,2825.25,NaN,NaN,NaN


In [24]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord_port.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord_port.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\1650912554.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord_port.csv')


### Quaterly hours for Macrozone Sud

In [25]:
import sys
print(sys.executable)

c:\Users\samueledelia\.conda\envs\myenv\python.exe


In [26]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Sud
qh_sud = qh[qh['MACROZONA'] == 'SUD']
qh_sud = qh_sud.drop(columns = ['MACROZONA'])

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [27]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_ENERGY_BALANCE" 
power = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power = power.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
power = power[['ENERGY_BALANCE_GWh', 'ORAINI', 'ENERGY_SOURCE']]
# Convert ORAINI to datetime using the correct method
power['ORAINI'] = pd.to_datetime(power['ORAINI'], format='%Y%m%d%H%M')
power.set_index('ORAINI', inplace=True)
thermo = utils.filter_and_rename_energy_source(power, 'Thermal', 'THERMO_GWH')
wind = utils.filter_and_rename_energy_source(power, 'Wind', 'WIND_GWH')
solar = utils.filter_and_rename_energy_source(power, 'Photovoltaic', 'SOLAR_GWH')
hydro = utils.filter_and_rename_energy_source(power, 'Hydro', 'HYDRO_GWH')
geothermal = utils.filter_and_rename_energy_source(power, 'Geothermal', 'GEO_GWH')
forex = utils.filter_and_rename_energy_source(power, 'Net Foreign Exchange', 'FOREX_GWH')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [28]:
# Resample the data to 15-minute intervals and divide by 4
wind = wind.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
wind['WIND_GWH'] = wind['WIND_GWH'] / 4
wind['WIND_MWH'] = wind['WIND_GWH'] * 1000
# Drop the original hourly GWh column
wind = wind.drop(columns=['WIND_GWH'])

# Resample the data to 15-minute intervals and divide by 4
solar = solar.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
solar['SOLAR_GWH'] = solar['SOLAR_GWH'] / 4
solar['SOLAR_MWH'] = solar['SOLAR_GWH'] * 1000
# Drop the original hourly GWh column
solar = solar.drop(columns=['SOLAR_GWH'])

# Resample the data to 15-minute intervals and divide by 4
thermo = thermo.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
thermo['THERMO_GWH'] = thermo['THERMO_GWH'] / 4
thermo['THERMO_MWH'] = thermo['THERMO_GWH'] * 1000
# Drop the original hourly GWh column
thermo = thermo.drop(columns=['THERMO_GWH'])

# Resample the data to 15-minute intervals and divide by 4
hydro = hydro.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
hydro['HYDRO_GWH'] = hydro['HYDRO_GWH'] / 4
hydro['HYDRO_MWH'] = hydro['HYDRO_GWH'] * 1000
# Drop the original hourly GWh column
hydro = hydro.drop(columns=['HYDRO_GWH'])

# Resample the data to 15-minute intervals and divide by 4
geothermal = geothermal.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
geothermal['GEO_GWH'] = geothermal['GEO_GWH'] / 4
geothermal['GEO_MWH'] = geothermal['GEO_GWH'] * 1000
# Drop the original hourly GWh column
geothermal = geothermal.drop(columns=['GEO_GWH'])

In [29]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_PHYSICAL_FOREIGN_FLOW" 
physical_forex = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
physical_forex = physical_forex.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
physical_forex = physical_forex[['PHYSICAL_FOREIGN_FLOW_MW', 'ORAINI', 'COUNTRY']]
# Convert ORAINI to datetime using the correct method
physical_forex['ORAINI'] = pd.to_datetime(physical_forex['ORAINI'], format='%Y%m%d%H%M')
physical_forex.set_index('ORAINI', inplace=True)


c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [30]:
austria_physical = utils.process_foreign_exchange(physical_forex, 'Austria')
corsica_physical = utils.process_foreign_exchange(physical_forex, 'Corsica')
france_physical = utils.process_foreign_exchange(physical_forex, 'France')
greece_physical = utils.process_foreign_exchange(physical_forex, 'Greece')
malta_physical = utils.process_foreign_exchange(physical_forex, 'Malta')
slovenia_physical = utils.process_foreign_exchange(physical_forex, 'Slovenia')
switzerland_physical = utils.process_foreign_exchange(physical_forex, 'Switzerland')
montenegno_physical = utils.process_foreign_exchange(physical_forex, 'Montenegno')

In [31]:
montenegno_physical

,,MONTENEGNO_MWQH
ORAINI,ORAINI,


In [32]:
corsica_physical

,CORSICA_MWQH
ORAINI,
2019-01-01 00:00:00,-35.5
2019-01-01 00:15:00,-35.5
2019-01-01 00:30:00,-35.5
2019-01-01 00:45:00,-35.5
2019-01-01 01:00:00,-36.5
...,...
2024-12-17 22:00:00,-32.0
2024-12-17 22:15:00,-32.0
2024-12-17 22:30:00,-32.0


In [33]:
greece_physical

,GREECE_MWQH
ORAINI,
2019-01-01 00:00:00,-78.25
2019-01-01 00:15:00,-78.25
2019-01-01 00:30:00,-78.25
2019-01-01 00:45:00,-78.25
2019-01-01 01:00:00,-75.25
...,...
2024-12-17 22:00:00,-59.00
2024-12-17 22:15:00,-59.00
2024-12-17 22:30:00,-59.00


In [34]:
malta_physical

,MALTA_MWQH
ORAINI,
2019-01-01 00:00:00,-20.00
2019-01-01 00:15:00,-20.00
2019-01-01 00:30:00,-20.00
2019-01-01 00:45:00,-20.00
2019-01-01 01:00:00,-17.50
...,...
2024-12-17 22:00:00,-36.75
2024-12-17 22:15:00,-36.75
2024-12-17 22:30:00,-36.75


In [35]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME,FLOWQUARTER,TIMERESOLUTION
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097,NaN,None
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097,NaN,None
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097,NaN,None
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100,NaN,None
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-13 19:00:00,43210.855,44069.484,3675.566,7793.277,23323.551,1174.003,2364.969,2613.840,0.0,None,...,477.000,0.0,0.0,0.0,0.0,44010.515,43510.555,2024-12-12 13:10:49.317,20.0,PT60
2024-12-13 20:00:00,40943.994,40943.994,3403.500,7425.759,21669.275,1153.128,2320.965,2515.856,0.0,None,...,784.000,0.0,0.0,0.0,625.0,40172.271,40943.994,2024-12-12 13:10:49.317,21.0,PT60
2024-12-13 21:00:00,37181.938,37181.938,3108.442,6828.312,20012.009,1075.712,2164.911,2326.693,0.0,None,...,1623.000,0.0,0.0,0.0,533.0,36149.383,37181.938,2024-12-12 13:10:49.317,22.0,PT60


In [36]:
print(mgp_volumes.columns)

Index(['TOTAL_PURCHASES', 'NAT_PURCHASES', 'CNOR_PURCHASES', 'CSUD_PURCHASES',
       'NORD_PURCHASES', 'SARD_PURCHASES', 'SICI_PURCHASES', 'SUD_PURCHASES',
       'AUST_PURCHASES', 'BRNN_PURCHASES', 'COAC_PURCHASES', 'CORS_PURCHASES',
       'FOGN_PURCHASES', 'FRAN_PURCHASES', 'GREC_PURCHASES', 'MFTV_PURCHASES',
       'PRGP_PURCHASES', 'ROSN_PURCHASES', 'SLOV_PURCHASES', 'SVIZ_PURCHASES',
       'BSP_PURCHASES', 'MALT_PURCHASES', 'XAUS_PURCHASES', 'XFRA_PURCHASES',
       'TOTAL_SALES', 'NAT_SALES', 'CNOR_SALES', 'CSUD_SALES', 'NORD_SALES',
       'SARD_SALES', 'SICI_SALES', 'SUD_SALES', 'AUST_SALES', 'BRNN_SALES',
       'COAC_SALES', 'CORS_SALES', 'FOGN_SALES', 'FRAN_SALES', 'GREC_SALES',
       'MFTV_SALES', 'PRGP_SALES', 'ROSN_SALES', 'SLOV_SALES', 'SVIZ_SALES',
       'BSP_SALES', 'MALT_SALES', 'XAUS_SALES', 'XFRA_SALES',
       'TOTITABSP_SALES', 'TOTITABSP_PURCHASES', 'LOADTIME', 'FLOWQUARTER',
       'TIMERESOLUTION'],
      dtype='object')


In [37]:
print(mi1_volumes.columns)

Index(['TOTAL_PURCHASES', 'NAT_PURCHASES', 'CNOR_PURCHASES', 'CSUD_PURCHASES',
       'NORD_PURCHASES', 'SARD_PURCHASES', 'SICI_PURCHASES', 'SUD_PURCHASES',
       'AUST_PURCHASES', 'BRNN_PURCHASES', 'COAC_PURCHASES', 'CORS_PURCHASES',
       'FOGN_PURCHASES', 'FRAN_PURCHASES', 'GREC_PURCHASES', 'MFTV_PURCHASES',
       'PRGP_PURCHASES', 'ROSN_PURCHASES', 'SLOV_PURCHASES', 'SVIZ_PURCHASES',
       'BSP_PURCHASES', 'MALT_PURCHASES', 'XAUS_PURCHASES', 'XFRA_PURCHASES',
       'TOTAL_SALES', 'NAT_SALES', 'CNOR_SALES', 'CSUD_SALES', 'NORD_SALES',
       'SARD_SALES', 'SICI_SALES', 'SUD_SALES', 'AUST_SALES', 'BRNN_SALES',
       'COAC_SALES', 'CORS_SALES', 'FOGN_SALES', 'FRAN_SALES', 'GREC_SALES',
       'MFTV_SALES', 'PRGP_SALES', 'ROSN_SALES', 'SLOV_SALES', 'SVIZ_SALES',
       'BSP_SALES', 'MALT_SALES', 'XAUS_SALES', 'XFRA_SALES',
       'TOTITABSP_SALES', 'TOTITABSP_PURCHASES', 'LOADTIME', 'FLOWQUARTER',
       'TIMERESOLUTION'],
      dtype='object')


In [38]:
mgp_volumes_cnord = mgp_volumes[['CNOR_PURCHASES', 'CNOR_SALES']].copy()
mgp_volumes_cnord.rename(columns={"CNOR_PURCHASES": "MGP_CNORD_PURCHASES", "CNOR_SALES": "MGP_CNORD_SALES"}, inplace=True)
mgp_volumes_cnord = mgp_volumes_cnord[~mgp_volumes_cnord.index.duplicated(keep='first')]
mgp_volumes_cnord = mgp_volumes_cnord.resample('15min').ffill()/4

mgp_volumes_csud = mgp_volumes[['CSUD_PURCHASES', 'CSUD_SALES']].copy()
mgp_volumes_csud.rename(columns={"CSUD_PURCHASES": "MGP_CSUD_PURCHASES", "CSUD_SALES": "MGP_CSUD_SALES"}, inplace=True)
mgp_volumes_csud = mgp_volumes_csud[~mgp_volumes_csud.index.duplicated(keep='first')]
mgp_volumes_csud = mgp_volumes_csud.resample('15min').ffill()/4

mgp_volumes_sud = mgp_volumes[['SUD_PURCHASES', 'SUD_SALES']].copy()
mgp_volumes_sud.rename(columns={"SUD_PURCHASES": "MGP_SUD_PURCHASES", "SUD_SALES": "MGP_SUD_SALES"}, inplace=True)
mgp_volumes_sud = mgp_volumes_sud[~mgp_volumes_sud.index.duplicated(keep='first')]
mgp_volumes_sud = mgp_volumes_sud.resample('15min').ffill()/4

# Zone Sardegna (SARD)
mgp_volumes_sard = mgp_volumes[['SARD_PURCHASES', 'SARD_SALES']].copy()
mgp_volumes_sard.rename(columns={"SARD_PURCHASES": "MGP_SARD_PURCHASES", "SARD_SALES": "MGP_SARD_SALES"}, inplace=True)
mgp_volumes_sard = mgp_volumes_sard[~mgp_volumes_sard.index.duplicated(keep='first')]
mgp_volumes_sard = mgp_volumes_sard.resample('15min').ffill()/4

# Zone Sicilia (SICI)
mgp_volumes_sici = mgp_volumes[['SICI_PURCHASES', 'SICI_SALES']].copy()
mgp_volumes_sici.rename(columns={"SICI_PURCHASES": "MGP_SICI_PURCHASES", "SICI_SALES": "MGP_SICI_SALES"}, inplace=True)
mgp_volumes_sici = mgp_volumes_sici[~mgp_volumes_sici.index.duplicated(keep='first')]
mgp_volumes_sici = mgp_volumes_sici.resample('15min').ffill()/4

# Summing Purchases from all zones to get MSUD_PURCHASED
mgp_msud_purchased = (mgp_volumes_cnord['MGP_CNORD_PURCHASES'] + 
                  mgp_volumes_csud['MGP_CSUD_PURCHASES'] + 
                  mgp_volumes_sud['MGP_SUD_PURCHASES'] + 
                  mgp_volumes_sard['MGP_SARD_PURCHASES'] + 
                  mgp_volumes_sici['MGP_SICI_PURCHASES'])

# Summing Sales from all zones to get MSUD_SALES
mgp_msud_sales = (mgp_volumes_cnord['MGP_CNORD_SALES'] + 
              mgp_volumes_csud['MGP_CSUD_SALES'] + 
              mgp_volumes_sud['MGP_SUD_SALES'] + 
              mgp_volumes_sard['MGP_SARD_SALES'] + 
              mgp_volumes_sici['MGP_SICI_SALES'])

# Create DataFrame for MSUD_PURCHASED and MSUD_SALES
mgp_msud_volumes = pd.DataFrame({
    'MGP_MSUD_PURCHASED': mgp_msud_purchased,
    'MGP_MSUD_SALES': mgp_msud_sales
})

# MI1 for CNORD
mi1_volumes_cnord = mi1_volumes[['CNOR_PURCHASES', 'CNOR_SALES']].copy()
mi1_volumes_cnord.rename(columns={"CNOR_PURCHASES": "MI1_CNORD_PURCHASES", "CNOR_SALES": "MI1_CNORD_SALES"}, inplace=True)
mi1_volumes_cnord = mi1_volumes_cnord[~mi1_volumes_cnord.index.duplicated(keep='first')]
mi1_volumes_cnord = mi1_volumes_cnord.resample('15min').ffill()/4

# MI1 for CSUD
mi1_volumes_csud = mi1_volumes[['CSUD_PURCHASES', 'CSUD_SALES']].copy()
mi1_volumes_csud.rename(columns={"CSUD_PURCHASES": "MI1_CSUD_PURCHASES", "CSUD_SALES": "MI1_CSUD_SALES"}, inplace=True)
mi1_volumes_csud = mi1_volumes_csud[~mi1_volumes_csud.index.duplicated(keep='first')]
mi1_volumes_csud = mi1_volumes_csud.resample('15min').ffill()/4

# MI1 for SUD
mi1_volumes_sud = mi1_volumes[['SUD_PURCHASES', 'SUD_SALES']].copy()
mi1_volumes_sud.rename(columns={"SUD_PURCHASES": "MI1_SUD_PURCHASES", "SUD_SALES": "MI1_SUD_SALES"}, inplace=True)
mi1_volumes_sud = mi1_volumes_sud[~mi1_volumes_sud.index.duplicated(keep='first')]
mi1_volumes_sud = mi1_volumes_sud.resample('15min').ffill()/4

# MI1 for Sardegna (SARD)
mi1_volumes_sard = mi1_volumes[['SARD_PURCHASES', 'SARD_SALES']].copy()
mi1_volumes_sard.rename(columns={"SARD_PURCHASES": "MI1_SARD_PURCHASES", "SARD_SALES": "MI1_SARD_SALES"}, inplace=True)
mi1_volumes_sard = mi1_volumes_sard[~mi1_volumes_sard.index.duplicated(keep='first')]
mi1_volumes_sard = mi1_volumes_sard.resample('15min').ffill()/4

# MI1 for Sicilia (SICI)
mi1_volumes_sici = mi1_volumes[['SICI_PURCHASES', 'SICI_SALES']].copy()
mi1_volumes_sici.rename(columns={"SICI_PURCHASES": "MI1_SICI_PURCHASES", "SICI_SALES": "MI1_SICI_SALES"}, inplace=True)
mi1_volumes_sici = mi1_volumes_sici[~mi1_volumes_sici.index.duplicated(keep='first')]
mi1_volumes_sici = mi1_volumes_sici.resample('15min').ffill()/4

# Summing Purchases from all MI1 zones to get MSUD_PURCHASED for MI1
mi1_msud_purchased = (mi1_volumes_cnord['MI1_CNORD_PURCHASES'] + 
                      mi1_volumes_csud['MI1_CSUD_PURCHASES'] + 
                      mi1_volumes_sud['MI1_SUD_PURCHASES'] + 
                      mi1_volumes_sard['MI1_SARD_PURCHASES'] + 
                      mi1_volumes_sici['MI1_SICI_PURCHASES'])

# Summing Sales from all MI1 zones to get MSUD_SALES for MI1
mi1_msud_sales = (mi1_volumes_cnord['MI1_CNORD_SALES'] + 
                  mi1_volumes_csud['MI1_CSUD_SALES'] + 
                  mi1_volumes_sud['MI1_SUD_SALES'] + 
                  mi1_volumes_sard['MI1_SARD_SALES'] + 
                  mi1_volumes_sici['MI1_SICI_SALES'])

# Create DataFrame for MSUD_PURCHASED and MSUD_SALES for MI1
mi1_msud_volumes = pd.DataFrame({
    'MI1_MSUD_PURCHASED': mi1_msud_purchased,
    'MI1_MSUD_SALES': mi1_msud_sales
})

In [39]:
mi1_msud_volumes

,MI1_MSUD_PURCHASED,MI1_MSUD_SALES
ORAINI,,
2019-01-01 00:00:00,73.74900,43.7800
2019-01-01 00:15:00,73.74900,43.7800
2019-01-01 00:30:00,73.74900,43.7800
2019-01-01 00:45:00,73.74900,43.7800
2019-01-01 01:00:00,70.33575,18.5275
...,...,...
2024-12-13 22:00:00,208.63275,110.7180
2024-12-13 22:15:00,208.63275,110.7180
2024-12-13 22:30:00,208.63275,110.7180


In [40]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SCHEDULED_INTERNAL_EXCHANGE"
scheduled_internal = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
scheduled_internal = scheduled_internal.sort_values(by='ORAINI')

# Filter the DataFrame for the relevant columns and rows
scheduled_internal = scheduled_internal[['SCHEDULED_INTERNAL_EXCHANGE_MW', 'ORAINI', 'FLOW_FROM', 'FLOW_TO']]
# Convert ORAINI to datetime using the correct method
scheduled_internal['ORAINI'] = pd.to_datetime(scheduled_internal['ORAINI'], format='%Y%m%d%H%M')
scheduled_internal.set_index('ORAINI', inplace=True)
scheduled_internal = scheduled_internal[(scheduled_internal['FLOW_FROM'] == 'CNOR') & (scheduled_internal['FLOW_TO'] == 'NORD') ]
scheduled_internal = scheduled_internal.drop(columns=['FLOW_FROM','FLOW_TO'])
scheduled_internal_nord = scheduled_internal.resample('15min').ffill()/4

scheduled_internal_nord

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,SCHEDULED_INTERNAL_EXCHANGE_MW
ORAINI,
2019-01-01 00:00:00,158.7875
2019-01-01 00:15:00,158.7875
2019-01-01 00:30:00,158.7875
2019-01-01 00:45:00,158.7875
2019-01-01 01:00:00,235.1475
...,...
2024-12-17 11:00:00,140.4525
2024-12-17 11:15:00,140.4525
2024-12-17 11:30:00,140.4525


In [41]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_TOTAL_LOAD_QH" 
total_load = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
total_load = total_load.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
total_load = total_load[['ZONA', 'TOTAL_LOAD_MW', 'FORECAST_TOTAL_LOAD_MW', 'ORAINI']]
# Convert ORAINI to datetime using the correct method
total_load['ORAINI'] = pd.to_datetime(total_load['ORAINI'], format='%Y%m%d%H%M')
total_load.set_index('ORAINI', inplace=True)
subset_zonas = ['CNORD', 'CSUD', 'SUD', 'SICI', 'SARD', 'CALA']
msud_load = utils.filter_load_by_zona(total_load, subset_zonas)

# Group by datetime and sum TOTAL_LOAD_MW and FORECAST_TOTAL_LOAD_MW
msud_load = msud_load.groupby(msud_load.index).agg(
    TOTAL_LOAD_MW=('TOTAL_LOAD_MW', 'sum'),
    FORECAST_TOTAL_LOAD_MW=('FORECAST_TOTAL_LOAD_MW', 'sum')
).reset_index()

msud_load.set_index('ORAINI', inplace=True)

msud_load

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,9932.667,10193.110
2019-01-01 00:15:00,9788.694,10028.134
2019-01-01 00:30:00,9709.883,9886.046
2019-01-01 00:45:00,9610.960,9770.148
2019-01-01 01:00:00,9472.551,9690.910
...,...,...
2024-12-17 22:45:00,13014.683,13412.545
2024-12-17 23:00:00,12613.763,12975.906
2024-12-17 23:15:00,12447.515,12804.248


In [42]:
# Load the data
df_load_ensoe_path = r'C:\imbalance_forecast\data\df_load_entsoe.csv'
df_load_ensoe_path = pd.read_csv(df_load_ensoe_path, index_col=0, parse_dates=True)

# Access the relevant columns
msud_load_entsoe = df_load_ensoe_path[['DAY_AHEAD_TOTAL_LOAD_MW_MSUD', 'ACTUAL_TOTAL_LOAD_MW_MSUD']]

# Check for duplicate timestamps
duplicates = msud_load_entsoe.index.duplicated(keep=False)

if duplicates.any():
    # Handle duplicates: average values for duplicate timestamps
    msud_load_entsoe = msud_load_entsoe.groupby(msud_load_entsoe.index).mean()

# Now resample the data
msud_load_entsoe = msud_load_entsoe.resample('15min').ffill() / 4

# Display the result
msud_load_entsoe

,DAY_AHEAD_TOTAL_LOAD_MW_MSUD,ACTUAL_TOTAL_LOAD_MW_MSUD
ORAINI,,
2019-01-01 00:00:00,2989.00,3033.25
2019-01-01 00:15:00,2989.00,3033.25
2019-01-01 00:30:00,2989.00,3033.25
2019-01-01 00:45:00,2989.00,3033.25
2019-01-01 01:00:00,2871.00,2952.75
...,...,...
2024-11-27 06:00:00,3099.25,3221.50
2024-11-27 06:15:00,3099.25,3221.50
2024-11-27 06:30:00,3099.25,3221.50


In [43]:
lags = [-1, -4, -5, -96]

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(qh_sud['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged

,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag5,SBIL_MWH_lag96
ORAINI,,,,
2019-01-01 00:00:00,NaN,NaN,NaN,NaN
2019-01-01 00:15:00,-71.120,NaN,NaN,NaN
2019-01-01 00:30:00,-110.515,NaN,NaN,NaN
2019-01-01 00:45:00,-88.851,NaN,NaN,NaN
2019-01-01 01:00:00,-80.145,-71.120,NaN,NaN
...,...,...,...,...
2024-12-18 15:15:00,271.238,157.838,254.334,83.620
2024-12-18 15:30:00,134.574,2.240,157.838,-73.747
2024-12-18 15:45:00,24.842,-101.331,2.240,-190.886


In [44]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [qh_sud, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, corsica_physical, greece_physical, malta_physical, scheduled_internal_nord, mgp_msud_volumes, mi1_msud_volumes, msud_load, msud_load_entsoe]

# Use reduce to merge all DataFrames on 'ORAINI'
df_sud = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

df_sud = df_sud[df_sud.index <= '2024-03-26']
df_sud = df_sud[df_sud.index >= '2019-01-03']

df_sud.head(35)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag5,SBIL_MWH_lag96,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,...,MALTA_MWQH,SCHEDULED_INTERNAL_EXCHANGE_MW,MGP_MSUD_PURCHASED,MGP_MSUD_SALES,MI1_MSUD_PURCHASED,MI1_MSUD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_MSUD,ACTUAL_TOTAL_LOAD_MW_MSUD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-03 00:00:00,-192.663,-10.170,-361.370,-70.246,49.648,1402.5,3242.5,0.0,407.5,167.5,...,-21.00,82.8000,2885.25775,2787.31725,64.88725,101.53075,9995.598,10486.501,3051.75,3026.75
2019-01-03 00:15:00,-197.137,-192.663,-194.208,-361.370,53.289,1402.5,3242.5,0.0,407.5,167.5,...,-21.00,82.8000,2885.25775,2787.31725,64.88725,101.53075,9827.521,10357.940,3051.75,3026.75
2019-01-03 00:30:00,-114.185,-197.137,-116.469,-194.208,103.794,1402.5,3242.5,0.0,407.5,167.5,...,-21.00,82.8000,2885.25775,2787.31725,64.88725,101.53075,9472.379,10004.163,3051.75,3026.75
2019-01-03 00:45:00,-65.038,-114.185,-10.170,-116.469,152.134,1402.5,3242.5,0.0,407.5,167.5,...,-21.00,82.8000,2885.25775,2787.31725,64.88725,101.53075,9127.666,9702.822,3051.75,3026.75
2019-01-03 01:00:00,-203.596,-65.038,-192.663,-10.170,-6.497,1360.0,3160.0,0.0,395.0,170.0,...,-18.50,221.8950,2647.56075,2736.00375,131.95675,83.49325,8848.526,9497.905,2792.75,2747.75
2019-01-03 01:15:00,-220.797,-203.596,-197.137,-192.663,21.437,1360.0,3160.0,0.0,395.0,170.0,...,-18.50,221.8950,2647.56075,2736.00375,131.95675,83.49325,8931.646,9315.634,2792.75,2747.75
2019-01-03 01:30:00,-158.892,-220.797,-114.185,-197.137,80.028,1360.0,3160.0,0.0,395.0,170.0,...,-18.50,221.8950,2647.56075,2736.00375,131.95675,83.49325,8577.672,8883.615,2792.75,2747.75
2019-01-03 01:45:00,-99.764,-158.892,-65.038,-114.185,122.229,1360.0,3160.0,0.0,395.0,170.0,...,-18.50,221.8950,2647.56075,2736.00375,131.95675,83.49325,8375.709,8719.341,2792.75,2747.75
2019-01-03 02:00:00,-119.805,-99.764,-203.596,-65.038,-5.805,1220.0,3150.0,0.0,405.0,170.0,...,-17.00,298.3425,2526.23725,2739.32125,147.18850,95.97175,8297.680,8660.096,2637.75,2585.75


In [45]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_sud.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_sud.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\2331997114.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_sud.csv')


### Add E. portfolio

In [46]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = """
SELECT * FROM POWER_UNBALANCE
"""
power_curve = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power_curve = power_curve.sort_values(by='TIMESTAMP')

power_curve

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TIMESTAMP,UNBALANCE_kW,SOURCE_ZONE
0,2023-11-01 01:00:23.830,-629.990,EOLICO_CSUD
1,2023-11-01 01:00:23.830,-208.450,EOLICO_SUD
2,2023-11-01 01:00:23.830,-781.500,IDRO-NON-PROGRAMMABILE_H_CNOR
3,2023-11-01 01:00:23.830,104.480,IDRO-NON-PROGRAMMABILE_H_CSUD
4,2023-11-01 01:00:23.830,2238.280,IDRO-NON-PROGRAMMABILE_H_NORD
...,...,...,...
7052866,2024-12-18 16:47:42.973,-255.120,EOLICO_CSUD
7052865,2024-12-18 16:47:42.973,359.040,EOLICO_CALA
7052881,2024-12-18 16:47:42.973,-92.900,SOLARE_SICI
7052872,2024-12-18 16:47:42.973,21.290,IDRO-NON-PROGRAMMABILE_H24_CNOR


In [47]:
msud_power_curve = power_curve.pivot(index='TIMESTAMP', columns='SOURCE_ZONE', values='UNBALANCE_kW')

# Rename columns for clarity (optional, if needed)
msud_power_curve.columns = [f"UNBALANCE_{col}" for col in msud_power_curve.columns]

# Display the resulting DataFrame
msud_power_curve = msud_power_curve.resample('15min').sum()
#msud_power_curve = msud_power_curve.resample('H').sum()

msud_power_curve = msud_power_curve[["UNBALANCE_EOLICO_CALA", "UNBALANCE_EOLICO_CSUD", "UNBALANCE_EOLICO_SICI", "UNBALANCE_EOLICO_SUD", "UNBALANCE_IDRO-NON-PROGRAMMABILE-MACROSUD", "UNBALANCE_SOLARE_CNOR", "UNBALANCE_SOLARE_CSUD", "UNBALANCE_SOLARE_SARD", "UNBALANCE_SOLARE_SICI", "UNBALANCE_SOLARE_SUD"]]

msud_power_curve = msud_power_curve.rename_axis("ORAINI")
# Summing Purchases from all MI1 zones to get MSUD_PURCHASED for MI1
msud_eolico = (msud_power_curve['UNBALANCE_EOLICO_CALA'] + 
                      msud_power_curve['UNBALANCE_EOLICO_CSUD'] + 
                      msud_power_curve['UNBALANCE_EOLICO_SICI'] + 
                      msud_power_curve['UNBALANCE_EOLICO_SUD'] )

# Summing Sales from all MI1 zones to get MSUD_SALES for MI1
msud_solare = (msud_power_curve['UNBALANCE_SOLARE_CNOR'] + 
                  msud_power_curve['UNBALANCE_SOLARE_CSUD'] + 
                  msud_power_curve['UNBALANCE_SOLARE_SARD'] + 
                  msud_power_curve['UNBALANCE_SOLARE_SICI'] + 
                  msud_power_curve['UNBALANCE_SOLARE_SUD'])

# Create DataFrame for MSUD_PURCHASED and MSUD_SALES for MI1
msud_port_unbalance = pd.DataFrame({
    'UNBALANCE_EOLICO_MACROSUD': msud_eolico,
    'UNBALANCE_SOLAR_MACROSUD': msud_solare,
})

msud_port_unbalance['UNBALANCE_IDRO-NON-PROGRAMMABILE-MACROSUD'] = msud_power_curve['UNBALANCE_IDRO-NON-PROGRAMMABILE-MACROSUD']

In [48]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [qh_sud, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, corsica_physical, greece_physical, malta_physical, scheduled_internal_nord, mgp_msud_volumes, mi1_msud_volumes, msud_load, msud_load_entsoe, msud_port_unbalance]

# Use reduce to merge all DataFrames on 'ORAINI'
df_sud_port = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

df_sud_port = df_sud_port[df_sud_port.index <= '2024-03-26']
df_sud_port = df_sud_port[df_sud_port.index >= '2019-01-03']

df_sud_port.head(35)


,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag5,SBIL_MWH_lag96,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,...,MGP_MSUD_SALES,MI1_MSUD_PURCHASED,MI1_MSUD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_MSUD,ACTUAL_TOTAL_LOAD_MW_MSUD,UNBALANCE_EOLICO_MACROSUD,UNBALANCE_SOLAR_MACROSUD,UNBALANCE_IDRO-NON-PROGRAMMABILE-MACROSUD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-03 00:00:00,-192.663,-10.170,-361.370,-70.246,49.648,1402.5,3242.5,0.0,407.5,167.5,...,2787.31725,64.88725,101.53075,9995.598,10486.501,3051.75,3026.75,NaN,NaN,NaN
2019-01-03 00:15:00,-197.137,-192.663,-194.208,-361.370,53.289,1402.5,3242.5,0.0,407.5,167.5,...,2787.31725,64.88725,101.53075,9827.521,10357.940,3051.75,3026.75,NaN,NaN,NaN
2019-01-03 00:30:00,-114.185,-197.137,-116.469,-194.208,103.794,1402.5,3242.5,0.0,407.5,167.5,...,2787.31725,64.88725,101.53075,9472.379,10004.163,3051.75,3026.75,NaN,NaN,NaN
2019-01-03 00:45:00,-65.038,-114.185,-10.170,-116.469,152.134,1402.5,3242.5,0.0,407.5,167.5,...,2787.31725,64.88725,101.53075,9127.666,9702.822,3051.75,3026.75,NaN,NaN,NaN
2019-01-03 01:00:00,-203.596,-65.038,-192.663,-10.170,-6.497,1360.0,3160.0,0.0,395.0,170.0,...,2736.00375,131.95675,83.49325,8848.526,9497.905,2792.75,2747.75,NaN,NaN,NaN
2019-01-03 01:15:00,-220.797,-203.596,-197.137,-192.663,21.437,1360.0,3160.0,0.0,395.0,170.0,...,2736.00375,131.95675,83.49325,8931.646,9315.634,2792.75,2747.75,NaN,NaN,NaN
2019-01-03 01:30:00,-158.892,-220.797,-114.185,-197.137,80.028,1360.0,3160.0,0.0,395.0,170.0,...,2736.00375,131.95675,83.49325,8577.672,8883.615,2792.75,2747.75,NaN,NaN,NaN
2019-01-03 01:45:00,-99.764,-158.892,-65.038,-114.185,122.229,1360.0,3160.0,0.0,395.0,170.0,...,2736.00375,131.95675,83.49325,8375.709,8719.341,2792.75,2747.75,NaN,NaN,NaN
2019-01-03 02:00:00,-119.805,-99.764,-203.596,-65.038,-5.805,1220.0,3150.0,0.0,405.0,170.0,...,2739.32125,147.18850,95.97175,8297.680,8660.096,2637.75,2585.75,NaN,NaN,NaN


In [49]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_sud_port.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_sud_port.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\1973063920.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_sud_port.csv')


## Hourly for Macrozone Nord

In [50]:
import sys
print(sys.executable)

c:\Users\samueledelia\.conda\envs\myenv\python.exe


In [51]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_nord = qh[qh['MACROZONA'] == 'NORD']


# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour
h_nord = qh_nord.resample('H').sum()

# Clean the 'MACROZONA' column to retain only 'NORD'
h_nord['MACROZONA'] = h_nord['MACROZONA'].str[:4]

h_nord = h_nord.drop(columns = ['MACROZONA'])
# Display the first few rows of the resampled data
print(h_nord.head())

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\1801842306.py:28: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  h_nord = qh_nord.resample('H').sum()


                     SBIL_MWH
ORAINI                       
2019-01-01 00:00:00  -111.900
2019-01-01 01:00:00  -112.451
2019-01-01 02:00:00   -88.849
2019-01-01 03:00:00   127.792
2019-01-01 04:00:00   162.699


In [52]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_ENERGY_BALANCE" 
power = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power = power.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
power = power[['ENERGY_BALANCE_GWh', 'ORAINI', 'ENERGY_SOURCE']]
# Convert ORAINI to datetime using the correct method
power['ORAINI'] = pd.to_datetime(power['ORAINI'], format='%Y%m%d%H%M')
power.set_index('ORAINI', inplace=True)
thermo = utils.filter_and_rename_energy_source(power, 'Thermal', 'THERMO_GWH')*1000
wind = utils.filter_and_rename_energy_source(power, 'Wind', 'WIND_GWH')*1000
solar = utils.filter_and_rename_energy_source(power, 'Photovoltaic', 'SOLAR_GWH')
hydro = utils.filter_and_rename_energy_source(power, 'Hydro', 'HYDRO_GWH')
geothermal = utils.filter_and_rename_energy_source(power, 'Geothermal', 'GEO_GWH')
forex = utils.filter_and_rename_energy_source(power, 'Net Foreign Exchange', 'FOREX_GWH')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [53]:
wind['WIND_MWH'] = wind['WIND_GWH'] * 1000
# Drop the original hourly GWh column
wind = wind.drop(columns=['WIND_GWH'])

solar['SOLAR_MWH'] = solar['SOLAR_GWH'] * 1000
# Drop the original hourly GWh column
solar = solar.drop(columns=['SOLAR_GWH'])

thermo['THERMO_MWH'] = thermo['THERMO_GWH'] * 1000
# Drop the original hourly GWh column
thermo = thermo.drop(columns=['THERMO_GWH'])

hydro['HYDRO_MWH'] = hydro['HYDRO_GWH'] * 1000
# Drop the original hourly GWh column
hydro = hydro.drop(columns=['HYDRO_GWH'])

geothermal['GEO_MWH'] = geothermal['GEO_GWH'] * 1000
# Drop the original hourly GWh column
geothermal = geothermal.drop(columns=['GEO_GWH'])

In [54]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_PHYSICAL_FOREIGN_FLOW" 
physical_forex = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
physical_forex = physical_forex.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
physical_forex = physical_forex[['PHYSICAL_FOREIGN_FLOW_MW', 'ORAINI', 'COUNTRY']]
# Convert ORAINI to datetime using the correct method
physical_forex['ORAINI'] = pd.to_datetime(physical_forex['ORAINI'], format='%Y%m%d%H%M')
physical_forex.set_index('ORAINI', inplace=True)

physical_forex

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,PHYSICAL_FOREIGN_FLOW_MW,COUNTRY
ORAINI,,
2019-01-01 00:00:00,156.0,Austria
2019-01-01 00:00:00,-142.0,Corsica
2019-01-01 00:00:00,-321.0,France
2019-01-01 00:00:00,-313.0,Greece
2019-01-01 00:00:00,-80.0,Malta
...,...,...
2024-12-17 23:00:00,2889.0,France
2024-12-17 23:00:00,403.0,Greece
2024-12-17 23:00:00,-105.0,Malta


In [55]:
def process_and_resample(df, country):
    country_data = utils.process_foreign_exchange(df, country)
    
    # Reset index if it's a MultiIndex
    country_data = country_data.reset_index()
    
    # Set 'ORAINI' as the index again for resampling
    country_data.set_index('ORAINI', inplace=True)
    
    # Resample and sum the values by hour
    country_data = country_data.resample('H').sum()
    
    return country_data

# Example usage for multiple countries:
austria_physical = process_and_resample(physical_forex, 'Austria')
corsica_physical = process_and_resample(physical_forex, 'Corsica')
france_physical = process_and_resample(physical_forex, 'France')
greece_physical = process_and_resample(physical_forex, 'Greece')
malta_physical = process_and_resample(physical_forex, 'Malta')
slovenia_physical = process_and_resample(physical_forex, 'Slovenia')
switzerland_physical = process_and_resample(physical_forex, 'Switzerland')
#montenegno_physical = process_and_resample(physical_forex, 'Montenegno')

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\621014083.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\621014083.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\621014083.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\621014083.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\621014083.py:11: FutureWarning: 'H' is deprec

In [56]:
austria_physical

,AUSTRIA_MWQH
ORAINI,
2019-01-01 00:00:00,156.00
2019-01-01 01:00:00,157.00
2019-01-01 02:00:00,147.00
2019-01-01 03:00:00,176.00
2019-01-01 04:00:00,165.00
...,...
2024-12-17 19:00:00,325.00
2024-12-17 20:00:00,328.00
2024-12-17 21:00:00,327.00


In [57]:
france_physical

,FRANCE_MWQH
ORAINI,
2019-01-01 00:00:00,-321.00
2019-01-01 01:00:00,-255.00
2019-01-01 02:00:00,-503.00
2019-01-01 03:00:00,-165.00
2019-01-01 04:00:00,-297.00
...,...
2024-12-17 19:00:00,3238.00
2024-12-17 20:00:00,3122.00
2024-12-17 21:00:00,3233.00


In [58]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME,FLOWQUARTER,TIMERESOLUTION
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097,NaN,None
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097,NaN,None
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097,NaN,None
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100,NaN,None
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-13 19:00:00,43210.855,44069.484,3675.566,7793.277,23323.551,1174.003,2364.969,2613.840,0.0,None,...,477.000,0.0,0.0,0.0,0.0,44010.515,43510.555,2024-12-12 13:10:49.317,20.0,PT60
2024-12-13 20:00:00,40943.994,40943.994,3403.500,7425.759,21669.275,1153.128,2320.965,2515.856,0.0,None,...,784.000,0.0,0.0,0.0,625.0,40172.271,40943.994,2024-12-12 13:10:49.317,21.0,PT60
2024-12-13 21:00:00,37181.938,37181.938,3108.442,6828.312,20012.009,1075.712,2164.911,2326.693,0.0,None,...,1623.000,0.0,0.0,0.0,533.0,36149.383,37181.938,2024-12-12 13:10:49.317,22.0,PT60


In [59]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mgp_volumes_nord.rename(columns={"NORD_PURCHASES": "MGP_NORD_PURCHASES", "NORD_SALES": "MGP_NORD_SALES"}, inplace=True)
mgp_volumes_nord = mgp_volumes_nord[~mgp_volumes_nord.index.duplicated(keep='first')]


mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
mi1_volumes = mi1_volumes.drop(columns=['MARKET'])
mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mi1_volumes_nord.rename(columns={"NORD_PURCHASES": "MI1_NORD_PURCHASES", "NORD_SALES": "MI1_NORD_SALES"}, inplace=True)
mi1_volumes_nord = mi1_volumes_nord[~mi1_volumes_nord.index.duplicated(keep='first')]

In [60]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SCHEDULED_INTERNAL_EXCHANGE"
scheduled_internal = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
scheduled_internal = scheduled_internal.sort_values(by='ORAINI')

# Filter the DataFrame for the relevant columns and rows
scheduled_internal = scheduled_internal[['SCHEDULED_INTERNAL_EXCHANGE_MW', 'ORAINI', 'FLOW_FROM', 'FLOW_TO']]
# Convert ORAINI to datetime using the correct method
scheduled_internal['ORAINI'] = pd.to_datetime(scheduled_internal['ORAINI'], format='%Y%m%d%H%M')
scheduled_internal.set_index('ORAINI', inplace=True)
scheduled_internal = scheduled_internal[(scheduled_internal['FLOW_FROM'] == 'CNOR') & (scheduled_internal['FLOW_TO'] == 'NORD') ]
scheduled_internal_nord = scheduled_internal.drop(columns=['FLOW_FROM','FLOW_TO'])

scheduled_internal_nord

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,SCHEDULED_INTERNAL_EXCHANGE_MW
ORAINI,
2019-01-01 00:00:00,635.15
2019-01-01 01:00:00,940.59
2019-01-01 02:00:00,1372.39
2019-01-01 03:00:00,1617.07
2019-01-01 04:00:00,1921.50
...,...
2024-12-16 13:00:00,39.39
2024-12-17 09:00:00,187.13
2024-12-17 10:00:00,940.05


In [61]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_TOTAL_LOAD_QH" 
total_load = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
total_load = total_load.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
total_load = total_load[['ZONA', 'TOTAL_LOAD_MW', 'FORECAST_TOTAL_LOAD_MW', 'ORAINI']]
# Convert ORAINI to datetime using the correct method
total_load['ORAINI'] = pd.to_datetime(total_load['ORAINI'], format='%Y%m%d%H%M')
total_load.set_index('ORAINI', inplace=True)
subset_zonas = ['NORD']
mnord_load = utils.filter_load_by_zona(total_load, subset_zonas)
mnord_load = mnord_load.drop(columns=['ZONA'])
mnord_load = mnord_load.resample('H').sum()
mnord_load

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\3996377423.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  mnord_load = mnord_load.resample('H').sum()


,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,47011.513,48015.081
2019-01-01 01:00:00,45175.351,46163.263
2019-01-01 02:00:00,42444.995,43497.895
2019-01-01 03:00:00,39593.045,41131.292
2019-01-01 04:00:00,38363.091,39863.454
...,...,...
2024-12-17 19:00:00,101048.826,101592.507
2024-12-17 20:00:00,95232.247,96623.861
2024-12-17 21:00:00,88072.535,89619.057


In [62]:
# Load the data
df_load_ensoe_path = r'C:\imbalance_forecast\data\df_load_entsoe.csv'
df_load_ensoe_path = pd.read_csv(df_load_ensoe_path, index_col=0, parse_dates=True)

# Access the relevant columns
mnord_load_entsoe = df_load_ensoe_path[['DAY_AHEAD_TOTAL_LOAD_MW_NORD', 'ACTUAL_TOTAL_LOAD_MW_NORD']]

# Check for duplicate timestamps
duplicates = mnord_load_entsoe.index.duplicated(keep=False)

if duplicates.any():
    # Handle duplicates: average values for duplicate timestamps
    mnord_load_entsoe = mnord_load_entsoe.groupby(mnord_load_entsoe.index).mean()


# Display the result
mnord_load_entsoe

,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,
2019-01-01 00:00:00,11110.0,11511.0
2019-01-01 01:00:00,10675.0,11039.0
2019-01-01 02:00:00,10080.0,10430.0
2019-01-01 03:00:00,9793.0,9811.0
2019-01-01 04:00:00,9485.0,9474.0
...,...,...
2024-11-27 03:00:00,14686.0,13987.0
2024-11-27 04:00:00,14510.0,14330.0
2024-11-27 05:00:00,15826.0,15747.0


In [63]:
lags = [-1, -2, -3, -24]

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(h_nord['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged = df_sbil_lagged.resample('H').sum()

df_sbil_lagged

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\1260965639.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_sbil_lagged = df_sbil_lagged.resample('H').sum()


,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24
ORAINI,,,,
2019-01-01 00:00:00,0.000,0.000,0.000,0.000
2019-01-01 01:00:00,-111.900,0.000,0.000,0.000
2019-01-01 02:00:00,-112.451,-111.900,0.000,0.000
2019-01-01 03:00:00,-88.849,-112.451,-111.900,0.000
2019-01-01 04:00:00,127.792,-88.849,-112.451,0.000
...,...,...,...,...
2024-12-18 12:00:00,833.325,728.298,762.205,51.051
2024-12-18 13:00:00,985.019,833.325,728.298,-617.973
2024-12-18 14:00:00,560.154,985.019,833.325,-667.040


In [64]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [h_nord, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, austria_physical, france_physical, slovenia_physical, switzerland_physical, scheduled_internal_nord, mgp_volumes_nord, mi1_volumes_nord, mnord_load, mnord_load_entsoe]
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

#df_nord_h = df_nord.drop(columns="MACROZONA")
df_nord_h = df_nord[df_nord.index <= '2024-03-26']
df_nord_h = df_nord_h[df_nord_h.index >= '2019-01-03']
df_nord_h

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,...,SWITZERLAND_MWQH,SCHEDULED_INTERNAL_EXCHANGE_MW,MGP_NORD_PURCHASES,MGP_NORD_SALES,MI1_NORD_PURCHASES,MI1_NORD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-03 00:00:00,42.456,-139.764,74.840,59.962,393.510,5610000.0,12970000.0,0.0,1630.0,670.0,...,2104.0,331.20,12884.917,8015.170,607.792,426.218,51751.436,54622.162,12778.0,12505.0
2019-01-03 01:00:00,-55.240,42.456,-139.764,74.840,470.408,5440000.0,12640000.0,0.0,1580.0,680.0,...,1595.0,887.58,12306.486,7970.890,519.879,538.758,49201.920,51573.941,11819.0,11674.0
2019-01-03 02:00:00,-46.830,-55.240,42.456,-139.764,477.902,4880000.0,12600000.0,0.0,1620.0,680.0,...,1552.0,1193.37,11961.762,7909.788,434.321,425.139,48006.133,49749.742,11309.0,11301.0
2019-01-03 03:00:00,-114.916,-46.830,-55.240,42.456,568.411,4920000.0,12670000.0,0.0,1560.0,680.0,...,1387.0,1379.92,11917.025,7838.137,351.280,415.050,47896.579,48849.077,11032.0,11241.0
2019-01-03 04:00:00,-65.532,-114.916,-46.830,-55.240,623.556,5000000.0,12720000.0,0.0,1620.0,680.0,...,1499.0,1478.67,12128.131,7881.073,367.500,399.960,48760.644,48803.013,11109.0,11454.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-25 20:00:00,97.961,-305.523,-232.093,250.896,15.634,3610000.0,19020000.0,0.0,8140.0,620.0,...,2059.0,NaN,20642.302,17266.889,1197.739,437.694,90354.068,90066.849,20628.0,20358.0
2024-03-25 21:00:00,428.124,97.961,-305.523,-232.093,-101.576,3790000.0,15810000.0,0.0,6340.0,620.0,...,3147.0,NaN,19087.810,14108.839,587.628,400.103,83821.282,84330.372,19018.0,18691.0
2024-03-25 22:00:00,355.414,428.124,97.961,-305.523,13.201,3730000.0,13280000.0,0.0,4860.0,620.0,...,3620.0,NaN,17342.100,12025.250,505.939,440.194,75128.017,76531.483,17171.0,16784.0


In [65]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord_h.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord_h.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\3337115461.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord_h.csv')


### Add E. portfolio

In [66]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = """
SELECT * FROM POWER_UNBALANCE
"""
power_curve = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power_curve = power_curve.sort_values(by='TIMESTAMP')

power_curve


power_curve = power_curve.pivot(index='TIMESTAMP', columns='SOURCE_ZONE', values='UNBALANCE_kW')

# Rename columns for clarity (optional, if needed)
power_curve.columns = [f"UNBALANCE_{col}" for col in power_curve.columns]

# Display the resulting DataFrame
power_curve = power_curve.resample('H').sum()

power_curve = power_curve[["UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD", "UNBALANCE_IDRO-PROGRAMMABILE_NORD", "UNBALANCE_SOLARE_NORD"]]

power_curve = power_curve.rename_axis("ORAINI")

power_curve



from functools import reduce
# List of all the DataFrames to be merged
dataframes = [h_nord, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, austria_physical, france_physical, slovenia_physical, switzerland_physical, scheduled_internal_nord, mgp_volumes_nord, mi1_volumes_nord, mnord_load, mnord_load_entsoe, power_curve]
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord_h_port = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

df_nord_h_port= df_nord_h_port[df_nord_h_port.index <= '2024-11-26']
df_nord_h_port= df_nord_h_port[df_nord_h_port.index >= '2019-01-03']
df_nord_h_port.head(35)


from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord_h_port.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord_h_port.to_csv(filepath)

<>:51: SyntaxWarning: invalid escape sequence '\i'
<>:51: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\2016443510.py:51: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord_h_port.csv')
c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\2016443510.py:29: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  power_curve = power_curve.resample('H').sum()


# Hourly for Macrozone Sud

In [67]:
import sys
print(sys.executable)

c:\Users\samueledelia\.conda\envs\myenv\python.exe


In [68]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_sud = qh[qh['MACROZONA'] == 'SUD']


# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour
h_sud = qh_sud.resample('H').sum()

# Clean the 'MACROZONA' column to retain only 'NORD'
h_sud['MACROZONA'] = h_sud['MACROZONA'].str[:3]

h_sud = h_sud.drop(columns = ['MACROZONA'])
# Display the first few rows of the resampled data
print(h_sud.head())

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\2281386225.py:28: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  h_sud = qh_sud.resample('H').sum()


                     SBIL_MWH
ORAINI                       
2019-01-01 00:00:00  -350.631
2019-01-01 01:00:00  -980.001
2019-01-01 02:00:00 -1024.145
2019-01-01 03:00:00  -636.110
2019-01-01 04:00:00  -356.706


In [69]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_ENERGY_BALANCE" 
power = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power = power.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
power = power[['ENERGY_BALANCE_GWh', 'ORAINI', 'ENERGY_SOURCE']]
# Convert ORAINI to datetime using the correct method
power['ORAINI'] = pd.to_datetime(power['ORAINI'], format='%Y%m%d%H%M')
power.set_index('ORAINI', inplace=True)
thermo = utils.filter_and_rename_energy_source(power, 'Thermal', 'THERMO_GWH')*1000
wind = utils.filter_and_rename_energy_source(power, 'Wind', 'WIND_GWH')*1000
solar = utils.filter_and_rename_energy_source(power, 'Photovoltaic', 'SOLAR_GWH')
hydro = utils.filter_and_rename_energy_source(power, 'Hydro', 'HYDRO_GWH')
geothermal = utils.filter_and_rename_energy_source(power, 'Geothermal', 'GEO_GWH')
forex = utils.filter_and_rename_energy_source(power, 'Net Foreign Exchange', 'FOREX_GWH')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [70]:
wind['WIND_MWH'] = wind['WIND_GWH'] * 1000
# Drop the original hourly GWh column
wind = wind.drop(columns=['WIND_GWH'])

solar['SOLAR_MWH'] = solar['SOLAR_GWH'] * 1000
# Drop the original hourly GWh column
solar = solar.drop(columns=['SOLAR_GWH'])

thermo['THERMO_MWH'] = thermo['THERMO_GWH'] * 1000
# Drop the original hourly GWh column
thermo = thermo.drop(columns=['THERMO_GWH'])

hydro['HYDRO_MWH'] = hydro['HYDRO_GWH'] * 1000
# Drop the original hourly GWh column
hydro = hydro.drop(columns=['HYDRO_GWH'])

geothermal['GEO_MWH'] = geothermal['GEO_GWH'] * 1000
# Drop the original hourly GWh column
geothermal = geothermal.drop(columns=['GEO_GWH'])

In [71]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_PHYSICAL_FOREIGN_FLOW" 
physical_forex = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
physical_forex = physical_forex.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
physical_forex = physical_forex[['PHYSICAL_FOREIGN_FLOW_MW', 'ORAINI', 'COUNTRY']]
# Convert ORAINI to datetime using the correct method
physical_forex['ORAINI'] = pd.to_datetime(physical_forex['ORAINI'], format='%Y%m%d%H%M')
physical_forex.set_index('ORAINI', inplace=True)

physical_forex

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,PHYSICAL_FOREIGN_FLOW_MW,COUNTRY
ORAINI,,
2019-01-01 00:00:00,156.0,Austria
2019-01-01 00:00:00,-142.0,Corsica
2019-01-01 00:00:00,-321.0,France
2019-01-01 00:00:00,-313.0,Greece
2019-01-01 00:00:00,-80.0,Malta
...,...,...
2024-12-17 23:00:00,2889.0,France
2024-12-17 23:00:00,403.0,Greece
2024-12-17 23:00:00,-105.0,Malta


In [72]:
def process_and_resample(df, country):
    country_data = utils.process_foreign_exchange(df, country)
    
    # Reset index if it's a MultiIndex
    country_data = country_data.reset_index()
    
    # Set 'ORAINI' as the index again for resampling
    country_data.set_index('ORAINI', inplace=True)
    
    # Resample and sum the values by hour
    country_data = country_data.resample('H').sum()
    
    return country_data

# Example usage for multiple countries:
austria_physical = process_and_resample(physical_forex, 'Austria')
corsica_physical = process_and_resample(physical_forex, 'Corsica')
france_physical = process_and_resample(physical_forex, 'France')
greece_physical = process_and_resample(physical_forex, 'Greece')
malta_physical = process_and_resample(physical_forex, 'Malta')
slovenia_physical = process_and_resample(physical_forex, 'Slovenia')
switzerland_physical = process_and_resample(physical_forex, 'Switzerland')

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\3996691647.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\3996691647.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\3996691647.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\3996691647.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\3996691647.py:11: FutureWarning: 'H' is d

In [73]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME,FLOWQUARTER,TIMERESOLUTION
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097,NaN,None
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097,NaN,None
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097,NaN,None
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100,NaN,None
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-13 19:00:00,43210.855,44069.484,3675.566,7793.277,23323.551,1174.003,2364.969,2613.840,0.0,None,...,477.000,0.0,0.0,0.0,0.0,44010.515,43510.555,2024-12-12 13:10:49.317,20.0,PT60
2024-12-13 20:00:00,40943.994,40943.994,3403.500,7425.759,21669.275,1153.128,2320.965,2515.856,0.0,None,...,784.000,0.0,0.0,0.0,625.0,40172.271,40943.994,2024-12-12 13:10:49.317,21.0,PT60
2024-12-13 21:00:00,37181.938,37181.938,3108.442,6828.312,20012.009,1075.712,2164.911,2326.693,0.0,None,...,1623.000,0.0,0.0,0.0,533.0,36149.383,37181.938,2024-12-12 13:10:49.317,22.0,PT60


In [74]:
mgp_volumes_cnord = mgp_volumes[['CNOR_PURCHASES', 'CNOR_SALES']].copy()
mgp_volumes_cnord.rename(columns={"CNOR_PURCHASES": "MGP_CNORD_PURCHASES", "CNOR_SALES": "MGP_CNORD_SALES"}, inplace=True)
mgp_volumes_cnord = mgp_volumes_cnord[~mgp_volumes_cnord.index.duplicated(keep='first')]


mgp_volumes_csud = mgp_volumes[['CSUD_PURCHASES', 'CSUD_SALES']].copy()
mgp_volumes_csud.rename(columns={"CSUD_PURCHASES": "MGP_CSUD_PURCHASES", "CSUD_SALES": "MGP_CSUD_SALES"}, inplace=True)
mgp_volumes_csud = mgp_volumes_csud[~mgp_volumes_csud.index.duplicated(keep='first')]


mgp_volumes_sud = mgp_volumes[['SUD_PURCHASES', 'SUD_SALES']].copy()
mgp_volumes_sud.rename(columns={"SUD_PURCHASES": "MGP_SUD_PURCHASES", "SUD_SALES": "MGP_SUD_SALES"}, inplace=True)
mgp_volumes_sud = mgp_volumes_sud[~mgp_volumes_sud.index.duplicated(keep='first')]


# Zone Sardegna (SARD)
mgp_volumes_sard = mgp_volumes[['SARD_PURCHASES', 'SARD_SALES']].copy()
mgp_volumes_sard.rename(columns={"SARD_PURCHASES": "MGP_SARD_PURCHASES", "SARD_SALES": "MGP_SARD_SALES"}, inplace=True)
mgp_volumes_sard = mgp_volumes_sard[~mgp_volumes_sard.index.duplicated(keep='first')]


# Zone Sicilia (SICI)
mgp_volumes_sici = mgp_volumes[['SICI_PURCHASES', 'SICI_SALES']].copy()
mgp_volumes_sici.rename(columns={"SICI_PURCHASES": "MGP_SICI_PURCHASES", "SICI_SALES": "MGP_SICI_SALES"}, inplace=True)
mgp_volumes_sici = mgp_volumes_sici[~mgp_volumes_sici.index.duplicated(keep='first')]


# Summing Purchases from all zones to get MSUD_PURCHASED
mgp_msud_purchased = (mgp_volumes_cnord['MGP_CNORD_PURCHASES'] + 
                  mgp_volumes_csud['MGP_CSUD_PURCHASES'] + 
                  mgp_volumes_sud['MGP_SUD_PURCHASES'] + 
                  mgp_volumes_sard['MGP_SARD_PURCHASES'] + 
                  mgp_volumes_sici['MGP_SICI_PURCHASES'])

# Summing Sales from all zones to get MSUD_SALES
mgp_msud_sales = (mgp_volumes_cnord['MGP_CNORD_SALES'] + 
              mgp_volumes_csud['MGP_CSUD_SALES'] + 
              mgp_volumes_sud['MGP_SUD_SALES'] + 
              mgp_volumes_sard['MGP_SARD_SALES'] + 
              mgp_volumes_sici['MGP_SICI_SALES'])

# Create DataFrame for MSUD_PURCHASED and MSUD_SALES
mgp_msud_volumes = pd.DataFrame({
    'MGP_MSUD_PURCHASED': mgp_msud_purchased,
    'MGP_MSUD_SALES': mgp_msud_sales
})

# MI1 for CNORD
mi1_volumes_cnord = mi1_volumes[['CNOR_PURCHASES', 'CNOR_SALES']].copy()
mi1_volumes_cnord.rename(columns={"CNOR_PURCHASES": "MI1_CNORD_PURCHASES", "CNOR_SALES": "MI1_CNORD_SALES"}, inplace=True)
mi1_volumes_cnord = mi1_volumes_cnord[~mi1_volumes_cnord.index.duplicated(keep='first')]


# MI1 for CSUD
mi1_volumes_csud = mi1_volumes[['CSUD_PURCHASES', 'CSUD_SALES']].copy()
mi1_volumes_csud.rename(columns={"CSUD_PURCHASES": "MI1_CSUD_PURCHASES", "CSUD_SALES": "MI1_CSUD_SALES"}, inplace=True)
mi1_volumes_csud = mi1_volumes_csud[~mi1_volumes_csud.index.duplicated(keep='first')]


# MI1 for SUD
mi1_volumes_sud = mi1_volumes[['SUD_PURCHASES', 'SUD_SALES']].copy()
mi1_volumes_sud.rename(columns={"SUD_PURCHASES": "MI1_SUD_PURCHASES", "SUD_SALES": "MI1_SUD_SALES"}, inplace=True)
mi1_volumes_sud = mi1_volumes_sud[~mi1_volumes_sud.index.duplicated(keep='first')]


# MI1 for Sardegna (SARD)
mi1_volumes_sard = mi1_volumes[['SARD_PURCHASES', 'SARD_SALES']].copy()
mi1_volumes_sard.rename(columns={"SARD_PURCHASES": "MI1_SARD_PURCHASES", "SARD_SALES": "MI1_SARD_SALES"}, inplace=True)
mi1_volumes_sard = mi1_volumes_sard[~mi1_volumes_sard.index.duplicated(keep='first')]


# MI1 for Sicilia (SICI)
mi1_volumes_sici = mi1_volumes[['SICI_PURCHASES', 'SICI_SALES']].copy()
mi1_volumes_sici.rename(columns={"SICI_PURCHASES": "MI1_SICI_PURCHASES", "SICI_SALES": "MI1_SICI_SALES"}, inplace=True)
mi1_volumes_sici = mi1_volumes_sici[~mi1_volumes_sici.index.duplicated(keep='first')]


# Summing Purchases from all MI1 zones to get MSUD_PURCHASED for MI1
mi1_msud_purchased = (mi1_volumes_cnord['MI1_CNORD_PURCHASES'] + 
                      mi1_volumes_csud['MI1_CSUD_PURCHASES'] + 
                      mi1_volumes_sud['MI1_SUD_PURCHASES'] + 
                      mi1_volumes_sard['MI1_SARD_PURCHASES'] + 
                      mi1_volumes_sici['MI1_SICI_PURCHASES'])

# Summing Sales from all MI1 zones to get MSUD_SALES for MI1
mi1_msud_sales = (mi1_volumes_cnord['MI1_CNORD_SALES'] + 
                  mi1_volumes_csud['MI1_CSUD_SALES'] + 
                  mi1_volumes_sud['MI1_SUD_SALES'] + 
                  mi1_volumes_sard['MI1_SARD_SALES'] + 
                  mi1_volumes_sici['MI1_SICI_SALES'])

# Create DataFrame for MSUD_PURCHASED and MSUD_SALES for MI1
mi1_msud_volumes = pd.DataFrame({
    'MI1_MSUD_PURCHASED': mi1_msud_purchased,
    'MI1_MSUD_SALES': mi1_msud_sales
})

In [75]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SCHEDULED_INTERNAL_EXCHANGE"
scheduled_internal = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
scheduled_internal = scheduled_internal.sort_values(by='ORAINI')

# Filter the DataFrame for the relevant columns and rows
scheduled_internal = scheduled_internal[['SCHEDULED_INTERNAL_EXCHANGE_MW', 'ORAINI', 'FLOW_FROM', 'FLOW_TO']]
# Convert ORAINI to datetime using the correct method
scheduled_internal['ORAINI'] = pd.to_datetime(scheduled_internal['ORAINI'], format='%Y%m%d%H%M')
scheduled_internal.set_index('ORAINI', inplace=True)
scheduled_internal = scheduled_internal[(scheduled_internal['FLOW_FROM'] == 'CNOR') & (scheduled_internal['FLOW_TO'] == 'NORD') ]
scheduled_internal_nord = scheduled_internal.drop(columns=['FLOW_FROM','FLOW_TO'])

scheduled_internal_nord

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,SCHEDULED_INTERNAL_EXCHANGE_MW
ORAINI,
2019-01-01 00:00:00,635.15
2019-01-01 01:00:00,940.59
2019-01-01 02:00:00,1372.39
2019-01-01 03:00:00,1617.07
2019-01-01 04:00:00,1921.50
...,...
2024-12-16 13:00:00,39.39
2024-12-17 09:00:00,187.13
2024-12-17 10:00:00,940.05


In [76]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_TOTAL_LOAD_QH" 
total_load = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
total_load = total_load.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
total_load = total_load[['ZONA', 'TOTAL_LOAD_MW', 'FORECAST_TOTAL_LOAD_MW', 'ORAINI']]
# Convert ORAINI to datetime using the correct method
total_load['ORAINI'] = pd.to_datetime(total_load['ORAINI'], format='%Y%m%d%H%M')
total_load.set_index('ORAINI', inplace=True)
subset_zonas = ['CNORD', 'CSUD', 'SUD', 'SICI', 'SARD', 'CALA']
msud_load = utils.filter_load_by_zona(total_load, subset_zonas)
msud_load = msud_load.drop(columns=['ZONA'])
msud_load = msud_load.resample('H').sum()
msud_load

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\859003010.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  msud_load = msud_load.resample('H').sum()


,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,39042.204,39877.438
2019-01-01 01:00:00,37653.004,38476.157
2019-01-01 02:00:00,35666.699,36551.175
2019-01-01 03:00:00,33124.838,34412.147
2019-01-01 04:00:00,31785.496,33028.603
...,...,...
2024-12-17 19:00:00,69021.499,69396.081
2024-12-17 20:00:00,67006.704,67985.838
2024-12-17 21:00:00,61577.735,62659.056


In [77]:
# Load the data
df_load_ensoe_path = r'C:\imbalance_forecast\data\df_load_entsoe.csv'
df_load_ensoe_path = pd.read_csv(df_load_ensoe_path, index_col=0, parse_dates=True)

# Access the relevant columns
msud_load_entsoe = df_load_ensoe_path[['DAY_AHEAD_TOTAL_LOAD_MW_MSUD', 'ACTUAL_TOTAL_LOAD_MW_MSUD']]

# Check for duplicate timestamps
duplicates = msud_load_entsoe.index.duplicated(keep=False)

if duplicates.any():
    # Handle duplicates: average values for duplicate timestamps
    msud_load_entsoe = msud_load_entsoe.groupby(msud_load_entsoe.index).mean()


# Display the result
msud_load_entsoe

,DAY_AHEAD_TOTAL_LOAD_MW_MSUD,ACTUAL_TOTAL_LOAD_MW_MSUD
ORAINI,,
2019-01-01 00:00:00,11956.0,12133.0
2019-01-01 01:00:00,11484.0,11811.0
2019-01-01 02:00:00,10855.0,11170.0
2019-01-01 03:00:00,10134.0,10444.0
2019-01-01 04:00:00,9817.0,9985.0
...,...,...
2024-11-27 03:00:00,9760.0,9514.0
2024-11-27 04:00:00,9800.0,10164.0
2024-11-27 05:00:00,10474.0,10886.0


In [78]:
lags = [-1, -2, -24, -25]  

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(h_sud['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged = df_sbil_lagged.resample('H').sum()

df_sbil_lagged

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\3369969057.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_sbil_lagged = df_sbil_lagged.resample('H').sum()


,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag24,SBIL_MWH_lag25
ORAINI,,,,
2019-01-01 00:00:00,0.000,0.000,0.000,0.000
2019-01-01 01:00:00,-350.631,0.000,0.000,0.000
2019-01-01 02:00:00,-980.001,-350.631,0.000,0.000
2019-01-01 03:00:00,-1024.145,-980.001,0.000,0.000
2019-01-01 04:00:00,-636.110,-1024.145,0.000,0.000
...,...,...,...,...
2024-12-18 12:00:00,-97.076,521.337,39.303,183.279
2024-12-18 13:00:00,-477.947,-97.076,-163.331,39.303
2024-12-18 14:00:00,52.560,-477.947,408.737,-163.331


In [79]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [h_sud, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, corsica_physical, greece_physical, malta_physical, scheduled_internal_nord, mgp_msud_volumes, mi1_msud_volumes, msud_load, msud_load_entsoe]
# Use reduce to merge all DataFrames on 'ORAINI'
df_sud = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

#df_sud_h = df_nord.drop(columns="MACROZONA")
df_sud_h = df_sud[df_sud.index <= '2024-03-26']
df_sud_h = df_sud_h[df_sud_h.index >= '2019-01-03']
df_sud_h

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag24,SBIL_MWH_lag25,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,...,MALTA_MWQH,SCHEDULED_INTERNAL_EXCHANGE_MW,MGP_MSUD_PURCHASED,MGP_MSUD_SALES,MI1_MSUD_PURCHASED,MI1_MSUD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_MSUD,ACTUAL_TOTAL_LOAD_MW_MSUD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-03 00:00:00,-569.023,-682.217,-932.309,358.865,232.501,5610000.0,12970000.0,0.0,1630.0,670.0,...,-84.0,331.20,11541.031,11149.269,259.549,406.123,38423.164,40551.426,12207.0,12107.0
2019-01-03 01:00:00,-683.049,-569.023,-682.217,217.197,358.865,5440000.0,12640000.0,0.0,1580.0,680.0,...,-74.0,887.58,10590.243,10944.015,527.827,333.973,34733.553,36416.495,11171.0,10991.0
2019-01-03 02:00:00,-393.432,-683.049,-569.023,145.261,217.197,4880000.0,12600000.0,0.0,1620.0,680.0,...,-68.0,1193.37,10104.949,10957.285,588.754,383.887,32697.619,33885.958,10551.0,10343.0
2019-01-03 03:00:00,-337.406,-393.432,-683.049,216.093,145.261,4920000.0,12670000.0,0.0,1560.0,680.0,...,-67.0,1379.92,9929.531,10959.635,577.448,390.123,31889.396,32523.419,10230.0,10089.0
2019-01-03 04:00:00,-243.602,-337.406,-393.432,785.817,216.093,5000000.0,12720000.0,0.0,1620.0,680.0,...,-72.0,1478.67,9990.754,11002.365,671.454,527.258,32089.654,32117.750,10109.0,10239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-25 20:00:00,400.438,-438.065,-449.447,298.911,149.590,3610000.0,19020000.0,0.0,8140.0,620.0,...,-109.0,NaN,15507.858,11682.770,885.892,1583.651,57944.205,57759.925,15510.0,15462.0
2024-03-25 21:00:00,512.993,400.438,-438.065,-196.565,298.911,3790000.0,15810000.0,0.0,6340.0,620.0,...,-75.0,NaN,14248.706,11195.225,698.814,678.104,54187.507,54517.437,14343.0,14313.0
2024-03-25 22:00:00,-152.588,512.993,400.438,-510.594,-196.565,3730000.0,13280000.0,0.0,4860.0,620.0,...,-114.0,NaN,12906.248,10039.126,488.814,558.321,49505.887,50431.034,12963.0,12859.0


In [80]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_sud_h.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_sud_h.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\2546119520.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_sud_h.csv')


### Add E. portfolio

In [81]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = """
SELECT * FROM POWER_UNBALANCE
"""
power = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power = power.sort_values(by='TIMESTAMP')

power

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TIMESTAMP,UNBALANCE_kW,SOURCE_ZONE
0,2023-11-01 01:00:23.830,-629.990,EOLICO_CSUD
1,2023-11-01 01:00:23.830,-208.450,EOLICO_SUD
2,2023-11-01 01:00:23.830,-781.500,IDRO-NON-PROGRAMMABILE_H_CNOR
3,2023-11-01 01:00:23.830,104.480,IDRO-NON-PROGRAMMABILE_H_CSUD
4,2023-11-01 01:00:23.830,2238.280,IDRO-NON-PROGRAMMABILE_H_NORD
...,...,...,...
7052975,2024-12-18 16:53:42.973,-14.620,EOLICO_SICI
7052974,2024-12-18 16:53:42.973,-215.270,EOLICO_CSUD
7052973,2024-12-18 16:53:42.973,-31.200,EOLICO_CALA
7052981,2024-12-18 16:53:42.973,10.550,IDRO-NON-PROGRAMMABILE_H24_CNOR


In [82]:
msud_power_curve = power.pivot(index='TIMESTAMP', columns='SOURCE_ZONE', values='UNBALANCE_kW')

# Rename columns for clarity (optional, if needed)
msud_power_curve.columns = [f"UNBALANCE_{col}" for col in msud_power_curve.columns]

# Display the resulting DataFrame
#msud_power_curve = msud_power_curve.resample('15min').sum()
msud_power_curve = msud_power_curve.resample('H').sum()

msud_power_curve = msud_power_curve[["UNBALANCE_EOLICO_CALA", "UNBALANCE_EOLICO_CSUD", "UNBALANCE_EOLICO_SICI", "UNBALANCE_EOLICO_SUD", "UNBALANCE_IDRO-NON-PROGRAMMABILE-MACROSUD", "UNBALANCE_SOLARE_CNOR", "UNBALANCE_SOLARE_CSUD", "UNBALANCE_SOLARE_SARD", "UNBALANCE_SOLARE_SICI", "UNBALANCE_SOLARE_SUD"]]

msud_power_curve = msud_power_curve.rename_axis("ORAINI")

msud_power_curve

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\3146492127.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  msud_power_curve = msud_power_curve.resample('H').sum()


,UNBALANCE_EOLICO_CALA,UNBALANCE_EOLICO_CSUD,UNBALANCE_EOLICO_SICI,UNBALANCE_EOLICO_SUD,UNBALANCE_IDRO-NON-PROGRAMMABILE-MACROSUD,UNBALANCE_SOLARE_CNOR,UNBALANCE_SOLARE_CSUD,UNBALANCE_SOLARE_SARD,UNBALANCE_SOLARE_SICI,UNBALANCE_SOLARE_SUD
ORAINI,,,,,,,,,,
2023-11-01 01:00:00,0.00,-11836.96,0.00,-9885.640,0.00,0.000,0.000,0.00,0.00,0.000
2023-11-01 02:00:00,0.00,9075.50,0.00,1052.930,0.00,0.000,0.000,0.00,0.00,0.000
2023-11-01 03:00:00,0.00,9300.76,0.00,13287.240,0.00,0.000,0.000,0.00,0.00,0.000
2023-11-01 04:00:00,0.00,9968.71,0.00,14031.350,0.00,0.000,0.000,0.00,0.00,0.000
2023-11-01 05:00:00,0.00,18647.19,0.00,4583.640,0.00,0.000,0.000,0.00,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...
2024-12-18 12:00:00,-23058.90,-7969.07,-82.25,24471.640,303.33,-263490.490,7294.161,-247428.69,16996.88,107329.401
2024-12-18 13:00:00,-25002.16,-14182.45,-29.87,18958.392,-7084.82,-110239.650,33422.650,-132626.63,12580.75,100665.080
2024-12-18 14:00:00,-7973.50,-11225.08,1361.55,7832.145,24030.59,-4111.910,-13729.000,-47839.63,4601.56,88566.630


In [83]:
# Summing Purchases from all MI1 zones to get MSUD_PURCHASED for MI1
msud_eolico = (msud_power_curve['UNBALANCE_EOLICO_CALA'] + 
                      msud_power_curve['UNBALANCE_EOLICO_CSUD'] + 
                      msud_power_curve['UNBALANCE_EOLICO_SICI'] + 
                      msud_power_curve['UNBALANCE_EOLICO_SUD'] )

# Summing Sales from all MI1 zones to get MSUD_SALES for MI1
msud_solare = (msud_power_curve['UNBALANCE_SOLARE_CNOR'] + 
                  msud_power_curve['UNBALANCE_SOLARE_CSUD'] + 
                  msud_power_curve['UNBALANCE_SOLARE_SARD'] + 
                  msud_power_curve['UNBALANCE_SOLARE_SICI'] + 
                  msud_power_curve['UNBALANCE_SOLARE_SUD'])

# Create DataFrame for MSUD_PURCHASED and MSUD_SALES for MI1
msud_port_unbalance = pd.DataFrame({
    'UNBALANCE_EOLICO_MACROSUD': msud_eolico,
    'UNBALANCE_SOLAR_MACROSUD': msud_solare,
})

msud_port_unbalance['UNBALANCE_IDRO-NON-PROGRAMMABILE-MACROSUD'] = msud_power_curve['UNBALANCE_IDRO-NON-PROGRAMMABILE-MACROSUD']

In [84]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [h_sud, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, corsica_physical, greece_physical, malta_physical, scheduled_internal_nord, mgp_msud_volumes, mi1_msud_volumes, msud_load, msud_load_entsoe, msud_port_unbalance]
# Use reduce to merge all DataFrames on 'ORAINI'
df_sud_h_port = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

#df_sud_h = df_nord.drop(columns="MACROZONA")
df_sud_h_port = df_sud_h_port[df_sud_h_port.index <= '2024-03-26']
df_sud_h_port = df_sud_h_port[df_sud_h_port.index >= '2019-01-03']
df_sud_h_port

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag24,SBIL_MWH_lag25,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,...,MGP_MSUD_SALES,MI1_MSUD_PURCHASED,MI1_MSUD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_MSUD,ACTUAL_TOTAL_LOAD_MW_MSUD,UNBALANCE_EOLICO_MACROSUD,UNBALANCE_SOLAR_MACROSUD,UNBALANCE_IDRO-NON-PROGRAMMABILE-MACROSUD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-03 00:00:00,-569.023,-682.217,-932.309,358.865,232.501,5610000.0,12970000.0,0.0,1630.0,670.0,...,11149.269,259.549,406.123,38423.164,40551.426,12207.0,12107.0,NaN,NaN,NaN
2019-01-03 01:00:00,-683.049,-569.023,-682.217,217.197,358.865,5440000.0,12640000.0,0.0,1580.0,680.0,...,10944.015,527.827,333.973,34733.553,36416.495,11171.0,10991.0,NaN,NaN,NaN
2019-01-03 02:00:00,-393.432,-683.049,-569.023,145.261,217.197,4880000.0,12600000.0,0.0,1620.0,680.0,...,10957.285,588.754,383.887,32697.619,33885.958,10551.0,10343.0,NaN,NaN,NaN
2019-01-03 03:00:00,-337.406,-393.432,-683.049,216.093,145.261,4920000.0,12670000.0,0.0,1560.0,680.0,...,10959.635,577.448,390.123,31889.396,32523.419,10230.0,10089.0,NaN,NaN,NaN
2019-01-03 04:00:00,-243.602,-337.406,-393.432,785.817,216.093,5000000.0,12720000.0,0.0,1620.0,680.0,...,11002.365,671.454,527.258,32089.654,32117.750,10109.0,10239.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-25 20:00:00,400.438,-438.065,-449.447,298.911,149.590,3610000.0,19020000.0,0.0,8140.0,620.0,...,11682.770,885.892,1583.651,57944.205,57759.925,15510.0,15462.0,15092.364,21979.00,0.0
2024-03-25 21:00:00,512.993,400.438,-438.065,-196.565,298.911,3790000.0,15810000.0,0.0,6340.0,620.0,...,11195.225,698.814,678.104,54187.507,54517.437,14343.0,14313.0,-4736.790,22248.54,0.0
2024-03-25 22:00:00,-152.588,512.993,400.438,-510.594,-196.565,3730000.0,13280000.0,0.0,4860.0,620.0,...,10039.126,488.814,558.321,49505.887,50431.034,12963.0,12859.0,22489.350,21978.00,0.0


In [85]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_sud_h_port.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_sud_h_port.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_13912\3084621903.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_sud_h_port.csv')
